Generating plots that explore the basic, static LOPC model. This notebook is for generating publication-ready plots, which will be saved directly to the LaTeX directory. To follow the process of exploring the data, look at the other notebooks, e.g. [here](../basic_LOPC.ipynb].

In [ ]:
# computation
import sys
import warnings
from functools import partial
from pathlib import Path
import string

import dask
import holoviews as hv

# plotting
import hvplot.xarray
import lumapi
import numpy as np
import xarray as xr
import pandas as pd
from collections import defaultdict, OrderedDict

from bokeh.io import export_png, export_svg
from holoviews import dim, opts
import colorcet
import panel as pn
import panel.widgets as pnw
from multilayer_simulator.helpers.mixins import convert_wavelength_and_frequency

# import xyzpy as xyz
# from xyzpy.gen.combo_runner import multi_concat
from multilayer_simulator.lumerical_classes import (
    LumericalOscillator,
    format_stackfield,
    format_stackrt,
)
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options

In [ ]:
# use matplotlib because rendering bokeh to svg is broken
# but use bokeh because matplotlib GridSpace opts are broken
# I regret using holoviews
hv.extension("bokeh", "matplotlib", inline=False, case_sensitive_completion=True)

universal_opts = dict(fontscale=2, title="")
matplotlib_opts = dict(fig_inches=5, aspect=2, fig_latex=True)
bokeh_opts = dict(width=700, height=300)
default_opts = (
    universal_opts | bokeh_opts
)  # toggle depending on which backend is in use
# default_opts = universal_opts|matplotlib_opts  # toggle depending on which backend is in use
opts.defaults(
    opts.Curve(**default_opts),
    opts.Image(**default_opts),
    opts.QuadMesh(**default_opts),
    opts.VLine(**default_opts, line_width=4),
    opts.Overlay(**default_opts),
    opts.Layout(**default_opts),
    opts.GridSpace(
        **default_opts, show_legend=False, shared_xaxis=True, shared_yaxis=True
    ),
)

xarray_engine = "h5netcdf"

# SAVE_FORMAT = "svg"

In [ ]:
hv.extension("bokeh", inline=False, case_sensitive_completion=True)  # use matplotlib because rendering bokeh to svg is broken
pn.config.throttled = True  # don't update interactive plots until mouse is unclicked

# default_color_cycle = hv.Cycle("Colorblind")  # Ruth doesn't like the inclusion of yellow, which is fair enough
default_color_cycle = hv.Cycle(colorcet.glasbey_dark)
default_dash_cycle = hv.Cycle(["solid", "dashed", "dashdot", "dotted", "dotdash"])
universal_opts = dict(fontscale=2, title="")
matplotlib_opts = dict(fig_inches=5, aspect=2, fig_latex=True)
bokeh_opts = dict(width=700, height=300)
opts.defaults(opts.Curve(**universal_opts|bokeh_opts, color=default_color_cycle, line_width=1.5),
              opts.Image(**universal_opts|bokeh_opts,),
              opts.QuadMesh(**universal_opts|bokeh_opts,),
              opts.VLine(**universal_opts|bokeh_opts, line_width=4),
              opts.Scatter(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Slope(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Area(**universal_opts|bokeh_opts, color=default_color_cycle),
              opts.Overlay(**universal_opts|bokeh_opts),
              opts.Layout(**universal_opts|bokeh_opts),
              opts.GridSpace(**universal_opts|bokeh_opts),
              )

xarray_engine='h5netcdf'

In [ ]:
root = (
    Path.cwd().parent.parent.parent
)  # depth of parents depends on if this is running in JupyterLab or Notebook

In [ ]:
code_path = root / r"research"

In [ ]:
data_path = code_path / r"notebooks/data"

In [ ]:
archive_path = root / r"thesis/LaTeX/chapters/chapter_1"

In [ ]:
fig_path = archive_path / "fig_chapter_1"

In [ ]:
if not code_path in sys.path:
    sys.path.append(str(code_path))
from LOPC import LOPC
from LOPC.helpers import (  # combo_length,; estimate_combo_run_time,
    assign_derived_attrs,
    complex_elements,
    coordinate_string,
    enhancement_factor,
    find_optimum_coords,
    fix_bin_labels,
    indexer_from_dataset,
    integrate_da,
    linewidth_calculator,
    lopc_data,
    max_min_pos,
    mean_and_std,
    normalise_over_dim,
    plot_da,
    plot_field,
    plot_optimum_over_dim,
    plot_secondary,
    plot_var,
    pre_process_for_plots,
    restack,
    sel_or_integrate,
    spectrum,
    visualise_multilayer,
    vlines,
)

Turn on auto-archiving of cells and Holoviews outputs. See the user guide [here](https://holoviews.org/user_guide/Exporting_and_Archiving.html).

Might need to install `ipympl`.

EDIT: This does not work but I'm leaving this here so a future researcher can avoid the rabbithole I fell down.

In [ ]:
# # This is the idiomatic way to record all generated figures with holoviews
# # This does NOT work in JupyterLab: see https://github.com/holoviz/holoviews/issues/3570
# # This also does not work in Jupyter Notebook
# # It's just utterly broken

# hv.archive.auto(root=str(archive_path), export_name="fig_chapter_2")

Due to a bug that I still do not fully understand, `hv.save` raises
```
RuntimeError: Neither firefox and geckodriver nor a variant of chromium browser and chromedriver are available on system PATH. You can install the former with 'conda install -c conda-forge firefox geckodriver'.
```
The below code is part of a workaround. (The second part is to pass `web_driver` to `export_png(hv.render(fig), ..., webdriver=web_driver)` rather than `hv.save(fig)`, which is supposed to just work.)

In [ ]:
options = Options()
options.add_argument("-headless")
web_driver = Firefox(
    options=options,
)

# Load/define datasets

## Load LOPC dataset

In [ ]:
# # chunks for per-angle plots
# chunks = {
#     "frequency": 256,
#     "excitonic_layer_thickness": 16,
#     "passive_layer_thickness": 32,
#     "theta": 1,
#     "num_periods": 16,
# }

In [ ]:
# chunks for plotting or integrating over angle
chunks = {
    "frequency": 256,
    "excitonic_layer_thickness": 16,
    "passive_layer_thickness": 32,
    "theta": 16,
    "num_periods": 1,
}

In [ ]:
run_number = 2

ds = xr.open_mfdataset(
    data_path / f"run_{run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=chunks,
)

# add derived attrs
ds = assign_derived_attrs(
    ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

In [ ]:
np_run_number = 3

np_ds = xr.open_mfdataset(
    data_path / f"run_{np_run_number}/LOPC.nc",
    engine=xarray_engine,
    lock=False,
    chunks=chunks,
)

# add derived attrs
np_ds = assign_derived_attrs(
    np_ds, per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"]
)

## Load reference slab dataset

In [ ]:
# useful variables
total_excitonic_thicknesses = np.unique(ds.total_excitonic_thickness)
total_passive_thicknesses = np.unique(ds.total_passive_thickness)
total_thicknesses = np.unique(ds.total_thickness)

In [ ]:
ref = xr.open_mfdataset(
    data_path / f"run_{run_number}/ref.nc",
    engine=xarray_engine,
    lock=False,
)

Note: `period=False` is an important option because otherwise it ends up a coordinate of `total_excitonic_thickness` and causes a conflict after binary operations with `ds`.

In [ ]:
ref = assign_derived_attrs(
    ref,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
)

In [ ]:
# compressed reference slab without passive layer
crs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
)

In [ ]:
# filled reference slab
frs_1 = (
    ref.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_thickness")
)

### Load derived variables

In [ ]:
polarised_attrs = ["Rs", "Rp", "Ts", "Tp", "As", "Ap"]

In [ ]:
norm_1 = xr.open_mfdataset(
    data_path / f"run_{run_number}/norm_1.nc",
    chunks=chunks,
)

In [ ]:
norm_2 = xr.open_mfdataset(
    data_path / f"run_{run_number}/norm_2.nc",
    chunks=chunks,
    engine=xarray_engine,
    lock=False,
)

In [ ]:
norm_2 = assign_derived_attrs(
    norm_2, absorption=False, unpolarised=False, per_oscillator=False
)

In [ ]:
gb_tet = ds[polarised_attrs].groupby("total_excitonic_thickness")

# 'biology' absorptance enhancement factor: normalise by reference slab type 1: compressed reference slab w/o passive layer
diff_1 = gb_tet - crs_1

diff_1 = assign_derived_attrs(
    dataset=diff_1,
    unpolarised=True,
    absorption=False,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
gb_tt = ds[polarised_attrs].groupby("total_thickness")

# 'stuffed' difference factor: difference with reference slab type 2: filled reference slab
diff_2 = gb_tt - frs_1

diff_2 = assign_derived_attrs(
    dataset=diff_2,
    unpolarised=True,
    absorption=False,
    period=True,  # reset period to only depend on two dims
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
    N_tot=False,
    per_oscillator=["Rs", "Rp", "R", "Ts", "Tp", "T", "As", "Ap", "A"],
)

In [ ]:
ds_flat_spectrum = xr.open_dataset(
    data_path / f"run_{run_number}/LOPC_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ds_flat_spectrum = assign_derived_attrs(ds_flat_spectrum)

In [ ]:
ref_flat_spectrum = xr.open_dataset(
    data_path / f"run_{run_number}/ref_flat_spectrum.nc",
    engine=xarray_engine,
    lock=False,
)
ref_flat_spectrum = assign_derived_attrs(
    ref_flat_spectrum,
    period=False,
    total_excitonic_thickness=False,
    total_passive_thickness=False,
    total_thickness=False,
)

In [ ]:
# Compressed reference slab without passive layer
crs_1_flat_spectrum = (
    ref_flat_spectrum.sel(
        remove_last_layer=1,
        passive_layer_thickness=0,
        excitonic_layer_thickness=total_excitonic_thicknesses,
        drop=True,
    )
    .squeeze(drop=True)
    .rename(excitonic_layer_thickness="total_excitonic_thickness")
)

In [ ]:
norm_flat_spectrum = (
    ds_flat_spectrum.groupby("total_excitonic_thickness") / crs_1_flat_spectrum
)  # .drop_sel(excitonic_layer_thickness=0)

In [ ]:
norm_flat_spectrum = enhancement_factor(
    ds=ds_flat_spectrum, ref=crs_1_flat_spectrum, common_dim="total_excitonic_thickness"
)

## Restacking passive layer thickness to period

In [ ]:
restack_plt_to_period = partial(
    restack,
    start_idxs=["passive_layer_thickness", "excitonic_layer_thickness"],
    end_idxs=["period", "excitonic_layer_thickness"],
)

In [ ]:
restacked_ds = restack_plt_to_period(ds)

In [ ]:
restacked_np_ds = restack_plt_to_period(np_ds)

In [ ]:
restacked_norm_1 = restack_plt_to_period(norm_1)

In [ ]:
restacked_norm_2 = restack_plt_to_period(norm_2)

In [ ]:
restacked_diff_1 = restack_plt_to_period(diff_1)

In [ ]:
restacked_diff_2 = restack_plt_to_period(diff_2)

In [ ]:
restacked_ds_flat_spectrum = restack_plt_to_period(ds_flat_spectrum)

In [ ]:
restacked_norm_flat_spectrum = restack_plt_to_period(norm_flat_spectrum)

# Plots

## Pre-processing

In [ ]:
blue = hv.Cycle.default_cycles["default_colors"][0]
red = hv.Cycle.default_cycles["default_colors"][1]
yellow = hv.Cycle.default_cycles["default_colors"][2]
green = hv.Cycle.default_cycles["default_colors"][3]

In [ ]:
wavelengths_in_nanometres = np.linspace(480, 880, 256)
wavelengths = wavelengths_in_nanometres * 1e-9
frequencies = convert_wavelength_and_frequency(wavelengths)
angles = np.linspace(0, 86, 64)

In [ ]:
default_oscillator_params = {
    "N": 1e26,
    "permittivity": 2.2,
    "lorentz_resonance_wavelength": 680,
    "lorentz_linewidth": 7.5e13,
}

In [ ]:
unpolarised_RTA = ["R", "T", "A"]
s_polarised_RTA = ["Rs", "Ts", "As"]
p_polarised_RTA = ["Rp", "Tp", "Ap"]
reflectance = ["Rs", "Rp", "R"]
transmittance = ["Ts", "Tp", "T"]
absorptance = ["As", "Ap", "A"]
per_oscillator_RTA = ["R_per_oscillator", "T_per_oscillator", "A_per_oscillator"]

In [ ]:
# WARNING: all these datasets will be fundamentally changed after this cell, to the extent that it can't be run twice
# For consistency, keep important calculations in the preceding section!
ds = pre_process_for_plots(ds)
restacked_ds = pre_process_for_plots(restacked_ds)
restacked_np_ds = pre_process_for_plots(restacked_np_ds, strict=False)
ref = pre_process_for_plots(ref)
crs_1 = pre_process_for_plots(crs_1)
frs_1 = pre_process_for_plots(frs_1)
norm_1 = pre_process_for_plots(norm_1)
restacked_norm_1 = pre_process_for_plots(restacked_norm_1)
norm_2 = pre_process_for_plots(norm_2)
restacked_norm_2 = pre_process_for_plots(restacked_norm_2)
diff_1 = pre_process_for_plots(diff_1)
restacked_diff_1 = pre_process_for_plots(restacked_diff_1)
diff_2 = pre_process_for_plots(diff_2)
restacked_diff_2 = pre_process_for_plots(restacked_diff_2)
ds_flat_spectrum = pre_process_for_plots(ds_flat_spectrum)
restacked_ds_flat_spectrum = pre_process_for_plots(restacked_ds_flat_spectrum)
ref_flat_spectrum = pre_process_for_plots(ref_flat_spectrum)
crs_1_flat_spectrum = pre_process_for_plots(crs_1_flat_spectrum)
norm_flat_spectrum = pre_process_for_plots(norm_flat_spectrum)
restacked_norm_flat_spectrum = pre_process_for_plots(restacked_norm_flat_spectrum)

In [ ]:
period_dim = hv.Dimension("period", label="Λ", unit="nm")
wavelength_dim = hv.Dimension("wavelength", label="λ", unit="nm")
real_index_dim = hv.Dimension("n")
imag_index_dim = hv.Dimension("k", label="ϰ")
angle_dim = hv.Dimension("theta", label="θ", unit="°")
num_periods_dim = hv.Dimension("num_periods", label="N")
z_dim = hv.Dimension("z", unit="μm")
E_dim = hv.Dimension("E_squared", label=r"\(|E|^2\)", unit="a.u.")

## Useful functions

In [ ]:
def outline_hook(plot, element):
    "Plot hook for bold outlines."
    plot.handles["plot"].outline_line_color = "black"

In [ ]:
# calculate PBG positions
# n_eff is almost n_p=1.35, because d_e is very small
# hence lambda = 2*1.35*Lambda

In [ ]:
# better estimate is to use f = d_e/Lambda
def pbg_position(d1, period, n1, n2):
    f = d1/period
    return 2*np.sqrt(f*n1**2 + (1-f)*n2**2)*period

### Lorentz lines

I want some sort of metric for 'near the resonance' and 'far from the resonance'. The natural unit of distance in this instance is the linewidth. The linewidth is given in rad/s so there need to be some conversions to get the equivalent lines in the plots by wavelength, but they are roughly symmetrical around the peak wavelength.

Based on the plots of the refractive index below, I think I will consider 'near' to be 'within two linewidths', and 'far' to be 'at least four linewidths away'.

In [ ]:
# resonance_line = hv.VLine(680, label='LO resonance wavelength').opts(line_dash='dotted')

# Convert from rad/s to Hz
lorentz_linewidth_frequency = default_oscillator_params["lorentz_linewidth"] / (
    2 * np.pi
)

In [ ]:
def linewidth_calculator_factory(centre, linewidth):
    return partial(linewidth_calculator, centre=centre, linewidth=linewidth)

In [ ]:
lorentz_line_frequency = linewidth_calculator_factory(
    convert_wavelength_and_frequency(680e-9), lorentz_linewidth_frequency
)


def lorentz_line_wavelength(x=None):
    x = -x if x is not None else x
    return convert_wavelength_and_frequency(lorentz_line_frequency(x))

In [ ]:
def lorentz_vlines(x=0, scale=1, mode="wavelength", **kwargs):
    if mode == "wavelength":
        line_func = lorentz_line_wavelength
    elif mode == "frequency":
        line_func = lorentz_line_frequency
    else:
        raise TypeError(f"mode should be 'wavelength' or 'frequency', not {mode}")

    match x:
        case [*xs]:
            line_pos = [line_func(x) / scale for x in xs]
        case x:
            line_pos = line_func(x) / scale

    return vlines(line_pos, **kwargs)

### Plotting functions

#### Select a wavelength or wavelength range based on the distance from the resonance in linewidths.

In [ ]:
def select_lorentz_line(da, lorentz_line=0, window_radius=0):
    if window_radius == 0:
        wavelength = lorentz_line_wavelength(lorentz_line) * 1e9
        wavelength_sel_method = "nearest"
    else:
        wavelength = slice(
            lorentz_line_wavelength(lorentz_line - window_radius) * 1e9,
            lorentz_line_wavelength(lorentz_line + window_radius) * 1e9,
        )
        wavelength_sel_method = None
    da = da.sel(wavelength=wavelength, method=wavelength_sel_method)

    return da

#### Plot a comparison of the reflectance and absorptance of the LOPC with that of the reference slab.

In [ ]:
opts_R = [
    opts.Curve(color=blue, ylim=(0, 1)),
    opts.Image(cmap="viridis", clim=(0, 1)),
    opts.QuadMesh(cmap="viridis", clim=(0, 1)),
]


def plot_R(
    variable="R",
    dataset=None,
    label_field="long_name",
    label_append=None,
    **hvplot_kwargs
):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_R)
    return plot


# # test
# plot_R(dataset=restacked_ds.sel(period=200, excitonic_layer_thickness=20, num_periods=10).squeeze(), x="wavelength", y="theta").opts(cmap="cividis", clim=(None, None))

In [ ]:
opts_T = [
    opts.Curve(color=yellow, ylim=(0, 1)),
    opts.Image(cmap="cividis", clim=(0, 1)),
    opts.QuadMesh(cmap="cividis", clim=(0, 1)),
]


def plot_T(
    variable="T",
    dataset=None,
    label_field="long_name",
    label_append=None,
    **hvplot_kwargs
):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_T)
    return plot

In [ ]:
opts_A = [
    opts.Curve(color=red, ylim=(0, 1)),
    opts.Image(cmap="inferno", clim=(0, 1)),
    opts.QuadMesh(cmap="inferno", clim=(0, 1)),
]


def plot_A(
    variable="A",
    dataset=None,
    label_field="long_name",
    label_append=None,
    **hvplot_kwargs
):
    plot = plot_var(variable, dataset, label_field, label_append, **hvplot_kwargs)
    plot.opts(*opts_A)
    return plot

In [ ]:
def plot_vars_to_funcs(plot_vars):
    var_func_mapping = {
        "R": plot_R,
        "T": plot_T,
        "A": plot_A,
        "Rs": partial(plot_R, variable="Rs"),
        "Ts": partial(plot_T, variable="Ts"),
        "As": partial(plot_A, variable="As"),
        "Rp": partial(plot_R, variable="Rp"),
        "Tp": partial(plot_T, variable="Tp"),
        "Ap": partial(plot_A, variable="Ap"),
        "R_per_oscillator": partial(plot_R, variable="R_per_oscillator"),
        "T_per_oscillator": partial(plot_T, variable="T_per_oscillator"),
        "A_per_oscillator": partial(plot_A, variable="A_per_oscillator"),
        "Rs_per_oscillator": partial(plot_R, variable="Rs_per_oscillator"),
        "Ts_per_oscillator": partial(plot_T, variable="Ts_per_oscillator"),
        "As_per_oscillator": partial(plot_A, variable="As_per_oscillator"),
        "Rp_per_oscillator": partial(plot_R, variable="Rp_per_oscillator"),
        "Tp_per_oscillator": partial(plot_T, variable="Tp_per_oscillator"),
        "Ap_per_oscillator": partial(plot_A, variable="Ap_per_oscillator"),
    }

    plot_funcs = []
    for var in plot_vars:
        try:
            func = var_func_mapping[var]
        except KeyError:
            func = partial(plot_var, variable=var)
        plot_funcs.append(func)

    return plot_funcs

In [ ]:
# new version
def plot_RTA(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta,
    title="",
    include=["LOPC", "CRS_1"],
    plot_vars=["R", "T", "A"],
    label_override=None,
    label_append=None,
):
    label_field = None  # for debugging
    label_append = "" if label_append is None else label_append

    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plot_funcs = [
        partial(func, x="wavelength", label_field=label_field)
        for func in plot_vars_to_funcs(plot_vars)
    ]
    curves = []
    if "LOPC" in include:
        lopc_label = " (LOPC)" if label_override is None else label_override
        lopc_label += label_append
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = sel_or_integrate(lopc_sel, "theta", theta, normalisation=1)
        lopc_curves = [
            plot_func(dataset=lopc_sel, label_append=lopc_label).opts(line_dash="solid")
            for plot_func in plot_funcs
        ]
        curves += lopc_curves
    if "CRS_1" in include:
        crs_1_label = " (CRS)" if label_override is None else label_override
        crs_1_label += label_append
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = sel_or_integrate(crs_1_sel, "theta", theta, normalisation=1)
        crs_1_curves = [
            plot_func(dataset=crs_1_sel, label_append=crs_1_label).opts(
                line_dash="dashed"
            )
            for plot_func in plot_funcs
        ]
        curves += crs_1_curves
    if "FRS_1" in include:
        frs_1_label = " (FRS)" if label_override is None else label_override
        frs_1_label += label_append
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = sel_or_integrate(frs_1_sel, "theta", theta, normalisation=1)
        frs_1_curves = [
            plot_func(dataset=frs_1_sel, label_append=frs_1_label).opts(
                line_dash="dotted"
            )
            for plot_func in plot_funcs
        ]
        curves += frs_1_curves

    overlay = hv.Overlay(curves).opts(
        opts.Curve(
            ylim=(0, 1),
            ylabel="Intensity",
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N, theta=theta)}",
        ),
    )

    return overlay


# # test
# display(
#     plot_RTA(200, 40, 20, 0, "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(
#         legend_position="right"
#     )
# )

# display(
#     plot_RTA(
#         200,
#         40,
#         20,
#         (10, 50),
#         "test RA only\n",
#         include=["LOPC", "CRS_1", "FRS_1"],
#         plot_vars=["R", "A"],
#     ).opts(opts.Overlay(legend_position="right"))
# )

# display(
#     plot_RTA(
#         200,
#         40,
#         20,
#         75,
#         "test\n",
#         include=["LOPC"],
#         plot_vars=["R_per_oscillator", "A_per_oscillator"],
#         label_append=" test",
#         label_override="OVERRIDDEN",
#     ).opts(opts.Curve(ylim=(None, None)), opts.Overlay(legend_position="right"))
# )

#### Plot a comparison of normal incidence to integrated

In [ ]:
def plot_comparison(
    *comparison_params: tuple[dict, list["opts"]], plot_func=plot_RTA, **shared_params
):
    param_opts = [
        (shared_params | comp_params, comp_opts)
        for comp_params, comp_opts in comparison_params
    ]
    plots = [
        plot_func(**comp_params).opts(*comp_opts)
        for comp_params, comp_opts in param_opts
    ]
    return plots

In [ ]:
def compare_RTA(*args, opts_cycle=None, plot_func=plot_RTA, **shared_params):
    default_opts = [
        [opts.Curve(line_dash=style)]
        for style in ["solid", "dashed", "dotted", "dotdash", "dashdot"]
    ]
    opts_cycle = default_opts if opts_cycle is None else opts_cycle

    # comparison_params = list(zip(args, opts_cycle))

    plots = plot_comparison(
        *zip(args, opts_cycle), plot_func=plot_func, **shared_params
    )
    overlay = hv.Overlay(plots).opts(opts.Overlay(legend_position="right"))

    return overlay


# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
#     "include": ["LOPC"],
# }
# compare_RTA({"theta": (0, 75), "label_override": " (integrated)"}, {"theta": 0, "label_override": " (θ = 0)"}, **shared_params)

In [ ]:
compare_RTA_normal_vs_integrated = partial(
    compare_RTA,
    {"theta": (0, 45), "label_override": " (integrated)"},
    {"theta": 0, "label_override": " (θ = 0)"},
    include=["LOPC"],
)

# # test
# shared_params = {
#     "period": 250,
#     "excitonic_layer_thickness": 70,
#     "num_periods": 30,
# }
# compare_RTA_normal_vs_integrated(**shared_params)

#### Plot the RTA of the structures in 2D

In [ ]:
def plot_RTA_2D(
    period,
    excitonic_layer_thickness,
    num_periods,
    theta=(0, 75),
    title="",
    include=["LOPC", "CRS_1"],
):
    P = period
    t = excitonic_layer_thickness
    N = num_periods

    plots = []
    if "LOPC" in include:
        lopc_sel = restacked_ds.sel(
            period=P, excitonic_layer_thickness=t, num_periods=N
        ).squeeze()
        lopc_sel = lopc_sel.sel(theta=slice(*theta))
        plots.append(
            lopc_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (LOPC)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            lopc_sel["T"]
            .hvplot(
                kind="image", x="wavelength", y="theta", title="Transmittance (LOPC)"
            )
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            lopc_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (LOPC)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "CRS_1" in include:
        crs_1_sel = crs_1.sel(total_excitonic_thickness=t * N).squeeze()
        crs_1_sel = crs_1_sel.sel(theta=slice(*theta))
        plots.append(
            crs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (CRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            crs_1_sel["T"]
            .hvplot(
                kind="image", x="wavelength", y="theta", title="Transmittance (CRS)"
            )
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            crs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (CRS)")
            .opts(opts.Image(cmap="inferno"))
        )
    if "FRS_1" in include:
        frs_1_sel = frs_1.sel(total_thickness=(P + t) * N).squeeze()
        frs_1_sel = frs_1_sel.sel(theta=slice(*theta))
        plots.append(
            frs_1_sel["R"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Reflectance (FRS)")
            .opts(opts.Image(cmap="viridis"))
        )
        plots.append(
            frs_1_sel["T"]
            .hvplot(
                kind="image", x="wavelength", y="theta", title="Transmittance (FRS)"
            )
            .opts(opts.Image(cmap="cividis"))
        )
        plots.append(
            frs_1_sel["A"]
            .hvplot(kind="image", x="wavelength", y="theta", title="Absorptance (FRS)")
            .opts(opts.Image(cmap="inferno"))
        )

    layout = hv.Layout(plots).opts(
        opts.Image(
            clim=(0, 1),
            clabel="Intensity",
        ),
        opts.Layout(
            title=f"{title}{coordinate_string(period=P, excitonic_layer_thickness=t, num_periods=N)}",
        ),
    )

    return layout


# # test
# display(plot_RTA_2D(200, 40, 20, (0, 90), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

# display(plot_RTA_2D(200, 40, 20, (10, 50), "test\n", include=["LOPC", "CRS_1", "FRS_1"]).opts(opts.Image(frame_width=200)).cols(3))

#### Plot an enhancement factor.

In [ ]:
def plot_ef(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} enhancement factor")
        plot *= hv.HLine(1).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} enhancement factor",
            cmap="RdBu_r",
            clim=(0.5, 1.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot


# # test
# sel_1 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 30}
# sel_2 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10, "theta": 0}
# sel_3 = {"period": 200, "excitonic_layer_thickness": 40, "num_periods": 10}
# display(
#     (
#         plot_ef("As", restacked_norm_1, sel_1, "nearest", "test\n")
#         + plot_ef("As", restacked_norm_2, sel_2, title="test2\n")
#     ).cols(1)
# )
# display(
#     plot_ef("As", restacked_norm_1, sel_3, title="test3\n", x="theta", y="wavelength").opts(clim=(0, 2), cmap="RdBu_r")
# )

#### Plot a difference factor.

In [ ]:
def plot_df(
    variable,
    dataset,
    sel=None,
    sel_method=None,
    title="",
    *,
    x="wavelength",
    y=None,
):
    sel = {} if sel is None else sel
    da = dataset[variable].sel(**sel, method=sel_method).squeeze()
    if y is None:
        plot = da.hvplot(x=x, label=f"{variable} difference factor")
        plot *= hv.HLine(0).opts(line_dash="dotted")
    else:
        plot = da.hvplot(
            kind="image",
            x=x,
            y=y,
            label=f"{variable} difference factor",
            cmap="RdBu_r",
            clim=(-0.5, 0.5),
        )
    plot = plot.opts(
        opts.Curve(
            title=f"{title}{coordinate_string(**sel)}",
        ),
        opts.Overlay(
            title=f"{title}{coordinate_string(**sel)}",
        ),
    )

    return plot

#### Test plot_optimum_over_dim

In [ ]:
# foo, bar = plot_optimum_over_dim(restacked_ds.A.sel(theta=0, wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

In [ ]:
# foo, bar = plot_optimum_over_dim(integrate_da(restacked_ds.A, "theta", normalisation=1).sel(wavelength=660, method="nearest"), "period", "excitonic_layer_thickness", "num_periods", "max")

#### Find and plot the min or max over any dimension.

In [ ]:
def variable_to_string(variable, append=None):
    if variable.endswith("_per_oscillator"):
        variable = variable[0] + "po"
    if append:
        variable = variable + append
    return variable

In [ ]:
# print(variable_to_string("A"))
# print(variable_to_string("A_per_oscillator"))
# print(variable_to_string("A", append="df"))
# print(variable_to_string("A_per_oscillator", append="df"))

In [ ]:
def wrapped_2D_plot(
    variable,
    dataset,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle=None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
    save_figs=True,
    shade_window=True,
):
    plots = []
    append = None

    if str(dataset) == "restacked_ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_ds.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"
        append = None
        stored_ds = xr.open_mfdataset(
            data_path / f"run_{run_number}/opt_over_period_LOPC.nc",
            engine=xarray_engine,
            lock=False,
        )
        stored_tab = pd.read_csv(
            data_path / f"run_{run_number}/optimal_LOPC.csv",
            index_col=[
                "variable",
                "maxmin",
                "lorentz_line",
                "window_radius",
                "theta_start",
                "theta_stop",
            ],
        )

    if str(dataset) == "restacked_norm_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_norm_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"
        append = "ef"
        stored_ds = xr.open_mfdataset(
            data_path / f"run_{run_number}/opt_over_period_norm_1.nc",
            engine=xarray_engine,
            lock=False,
        )
        stored_tab = pd.read_csv(
            data_path / f"run_{run_number}/optimal_norm_1.csv",
            index_col=[
                "variable",
                "maxmin",
                "lorentz_line",
                "window_radius",
                "theta_start",
                "theta_stop",
            ],
        )

    if str(dataset) == "restacked_diff_1":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = restacked_diff_1.drop_sel({"excitonic_layer_thickness": 0})
        dim = "period"
        append = "df"
        stored_ds = xr.open_mfdataset(
            data_path / f"run_{run_number}/opt_over_period_diff_1.nc",
            engine=xarray_engine,
            lock=False,
        )
        stored_tab = pd.read_csv(
            data_path / f"run_{run_number}/optimal_diff.csv",
            index_col=[
                "variable",
                "maxmin",
                "lorentz_line",
                "window_radius",
                "theta_start",
                "theta_stop",
            ],
        )

    da = dataset[variable]

    try:
        optimum_coords = stored_tab.loc[
            variable, optimise, lorentz_line, window_radius, theta, integrate_angle
        ]
        plot_1 = (
            stored_ds[variable]
            .sel(
                optimise=optimise,
                lorentz_line=lorentz_line,
                window_radius=window_radius,
                theta_start=theta,
                theta_stop=integrate_angle,
            )
            .hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods")
        )
        plot_1 *= hv.Points(
            (optimum_coords["excitonic_layer_thickness"], optimum_coords["num_periods"])
        )
    except (KeyError, NameError):
        warnings.warn("stored data not found, computing from scratch")
        if not integrate_angle:
            da = da.sel(theta=theta, method="nearest")
        else:  # integrate_angle must be a float, so that (theta, integrate_angle) is a slice syntax
            da = da.sel(theta=slice(theta, integrate_angle))
            da = integrate_da(da, "theta", weighting=1, normalisation=1)

        if period_start < period_stop:
            da = da.sel(period=slice(period_start, period_stop))
        else:  # otherwise no data is selected and everything breaks
            da = da.sel(period=slice(period_start, None))
        da = select_lorentz_line(
            da, lorentz_line=lorentz_line, window_radius=window_radius
        )

        if window_radius != 0:
            da = integrate_da(da, "wavelength", weighting=1, normalisation=1)

        plot_1, optimum_coords = plot_optimum_over_dim(
            da,
            dim=dim,
            x="excitonic_layer_thickness",
            y="num_periods",
            optimise=optimise,
        )
    finally:
        da = select_lorentz_line(
            dataset, lorentz_line=lorentz_line, window_radius=window_radius
        )

        vline_locs = [0]

        if window_radius == 0:
            wavelength = float(da.wavelength)
            title = f"{optimise.capitalize()}imum {variable} at {wavelength:.0f} nm"
            if lorentz_line != 0:  # don't put two lines over the resonance
                vline_locs.append(lorentz_line)
        else:
            wavelength_start = float(da.wavelength[0])
            wavelength_stop = float(da.wavelength[-1])
            title = f"{optimise.capitalize()}imum integrated {variable} between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
            vline_locs.append(lorentz_line - window_radius)
            vline_locs.append(lorentz_line + window_radius)

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    th = (theta, integrate_angle) if integrate_angle else theta

    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    # shade the integration window:
    if window_radius != 0 and shade_window:
        window_vspan = hv.VSpan(
            lorentz_lines.VLine.II.x, lorentz_lines.VLine.III.x
        ).opts(
            opts.VSpan(color="gray", alpha=0.1)
        )  # shade the area between the second and third lines
        lorentz_lines *= window_vspan

    plot_1.opts(
        opts.QuadMesh(cmap=cmap, xlabel=r"\(d_e\ \text{(nm)}\)"),
        opts.Points(color="red"),
        opts.Overlay(
            # title=f"{title}\nOptimal period: {P:.0f}",
            title=""
        ),
    )

    plots.append(plot_1)
    if save_figs:
        variable = variable_to_string(variable, append=append)

        export_png(
            hv.render(plot_1.options(toolbar=None)),
            filename=fig_path
            / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_2D.png",
            webdriver=web_driver,
        )
        # hv.save(
        #     plot_1,
        #     filename=fig_path
        #     / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_2D",
        #     fmt=SAVE_FORMAT,
        #     toolbar=False,
        # )

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(legend_position="right", toolbar=None)),
                filename=fig_path
                / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_0.png",
                webdriver=web_driver,
            )

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(legend_position="right", toolbar=None)),
                filename=fig_path
                / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_i.png",
                webdriver=web_driver,
            )
            # hv.save(
            #     new_plot.options(legend_position="right", toolbar=None),
            #     filename=fig_path
            #     / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_i",
            #     fmt=SAVE_FORMAT,
            # )

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
        new_plot *= lorentz_lines

        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(toolbar=None)),
                filename=fig_path
                / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_0.png",
                webdriver=web_driver,
            )

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = plot_ef(variable="A", dataset=restacked_norm_1, sel=sel)
            new_plot *= lorentz_lines
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)
            # replaces the lines below
            #             # this should all get separated out into its own function
            #             crs_1_like_ds = crs_1.sel(
            #                 total_excitonic_thickness=ds.total_excitonic_thickness
            #             )

            #             ds_int = sel_or_integrate(ds, dim="theta", val=th)
            #             crs_1_int = sel_or_integrate(crs_1_like_ds, dim="theta", val=th)
            #             norm = ds_int / crs_1_int
            #             restacked_norm = norm.stack(multiperiod=['passive_layer_thickness', 'excitonic_layer_thickness']).set_index(multiperiod=['period', 'excitonic_layer_thickness']).unstack()

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = plot_ef(variable="A", dataset=restacked_norm, sel=sel)
            new_plot *= lorentz_lines
            sel["theta"] = th
            new_plot = new_plot.opts(
                opts.Overlay(
                    # title=f"{coordinate_string(**sel)}",
                    title="",
                )
            )
        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(toolbar=None)),
                filename=fig_path
                / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_i.png",
                webdriver=web_driver,
            )
            # hv.save(
            #     new_plot,
            #     filename=fig_path
            #     / f"opt_{variable}_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_i",
            #     fmt=SAVE_FORMAT,
            #     toolbar=False,
            # )

    print(f"optimum parameters: P={P}, t={t}, N={N}")
    return hv.Layout(plots).cols(1)

#### Plot the E-field, overlayed with the refractive index profile and layer boundaries.

In [ ]:
# sometimes this errors on the first call for some reason
fdtd = lumapi.FDTD()

oscillator = LumericalOscillator(fdtd)

In [ ]:
# plot_field(
#     680,
#     lumerical_session=fdtd,
#     oscillator=oscillator,
#     ri_lower=1.35,
#     ri_upper=1.6,
#     excitonic_layer_thickness=30,
#     passive_layer_thickness=210,
#     num_periods=10,
# ).opts(opts.VSpan(color="gray"))


def wrap_plot_field(
    wavelength,
    excitonic_layer_thickness,
    passive_layer_thickness,
    num_periods,
    theta=0,
    z_scale=1e-6,
):
    coords = {
        "λ": wavelength,
        "Excitonic layer thickness": excitonic_layer_thickness,
        "Passive layer thickness": passive_layer_thickness,
        "Number of periods": num_periods,
    }

    title = f"{coordinate_string(**coords)}"

    return plot_field(
        wavelength=wavelength,
        lumerical_session=fdtd,
        oscillator=oscillator,
        ri_lower=1.35,
        ri_upper=1.6,
        excitonic_layer_thickness=excitonic_layer_thickness,
        passive_layer_thickness=passive_layer_thickness,
        num_periods=num_periods,
        angles=theta,
        z_scale=z_scale,
        remove_last_layer=False
    ).opts(
        opts.Curve(ylim=(0, None)),
        opts.VSpan(color="gray"),
        opts.Overlay(title=title, xlabel="z", ylabel="E_squared"),
    )

In [ ]:
# # an example of what this can do
# pn.interact(
#     wrap_plot_field,
#     wavelength=(480, 880),
#     excitonic_layer_thickness=(10, 200),
#     passive_layer_thickness=(0, 300),
#     num_periods=(1, 50),
# )

## Angular shift in reflectance

### P = 300 (for best display of the PBG)

In [ ]:
period = 300
excitonic_layer_thickness = 50
passive_layer_thickness = period - excitonic_layer_thickness
num_periods = 20

In [ ]:
plots = {
    theta: compare_RTA(
        {"plot_vars": ["R"]},
        {"plot_vars": ["Rs"]},
        {"plot_vars": ["Rp"]},
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
        num_periods=num_periods,
        theta=theta,
        include=["LOPC"],
        label_override="",
    )
    for theta in [0, 15, 30, 45]
}

In [ ]:
grid = hv.GridSpace(plots, kdims=[angle_dim])*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Overlay(show_legend=False, show_grid=True))
grid[0].opts(opts.Overlay(show_legend=True, legend_position="left"))

grid.opts(
    opts.Curve(xticks=2, yticks=3, frame_height=120, frame_width=160, toolbar=None, hooks=[outline_hook]),
    opts.VLine(line_dash="dotted", color=yellow, ),
    opts.Overlay(fontscale=1.5),
    # opts.GridSpace(fontsize={"xlabel": 8}, toolbar=None),
    opts.GridSpace(fontscale=1.5, toolbar=None),
)

In [ ]:
hv.save(grid, filename=fig_path/"R_lambda_theta_1D", fmt="png", toolbar=None)

In [ ]:
# Brewster's angle
np.arctan(1.45/1.35)*(180/np.pi)

In [ ]:
plots = OrderedDict({
    var: plot_R(
        variable=var,
        dataset=restacked_ds.sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
            num_periods=num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["R", "Rs", "Rp"]
})

In [ ]:
grid = hv.GridSpace(plots, kdims=["Reflectance"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Image(colorbar=False))
grid["R"].opts(opts.Image(colorbar=True, colorbar_position="left"))

grid.opts(
    opts.Image(xticks=3, frame_height=120, frame_width=160, toolbar=None, fontscale=1.5),
    opts.VLine(line_dash="dotted", color="white"),
    opts.Overlay(fontscale=1.5),
    opts.GridSpace(fontscale=1.5, toolbar=None),
)

In [ ]:
hv.save(grid, filename=fig_path/"R_lambda_theta_2D", fmt="png", toolbar=None)

### P=100 (to understand what happens to the high-angle reflectance in the absence of the PBG)

#### elt=50 (some reflectance at high angles)

In [ ]:
period = 100
excitonic_layer_thickness = 50
passive_layer_thickness = period - excitonic_layer_thickness
num_periods = 20

In [ ]:
plots = {
    theta: compare_RTA(
        {"plot_vars": ["R"]},
        {"plot_vars": ["Rs"]},
        {"plot_vars": ["Rp"]},
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
        num_periods=num_periods,
        theta=theta,
        include=["LOPC"],
        label_override="",
    )
    for theta in [0, 15, 30, 45]
}

In [ ]:
grid = hv.GridSpace(plots, kdims=["θ"])*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Overlay(show_legend=False, show_grid=True))
grid[0].opts(opts.Overlay(show_legend=True, legend_position="left"))

grid.opts(opts.Curve(xticks=2, yticks=3, frame_height=120, frame_width=120), opts.VLine(line_dash="dotted", color=yellow), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
# hv.save(grid, filename=fig_path/"R_lambda_theta_1D", fmt="png", toolbar=None)

In [ ]:
# Brewster's angle
np.arctan(1.45/1.35)*(180/np.pi)

In [ ]:
plots = OrderedDict({
    var: plot_R(
        variable=var,
        dataset=restacked_ds.sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
            num_periods=num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["R", "Rs", "Rp"]
})

In [ ]:
grid = hv.GridSpace(plots, kdims=["Reflectance"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Image(colorbar=False))
grid["R"].opts(opts.Image(colorbar=True, colorbar_position="left"))

grid.opts(opts.Image(xticks=3, frame_height=120, frame_width=160), opts.VLine(line_dash="dotted", color="white"), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
# hv.save(grid, filename=fig_path/"R_lambda_theta_2D", fmt="png", toolbar=None)

#### elt=10 (no reflectance at high angles)

In [ ]:
period = 100
excitonic_layer_thickness = 10
passive_layer_thickness = period - excitonic_layer_thickness
num_periods = 20

In [ ]:
plots = {
    theta: compare_RTA(
        {"plot_vars": ["R"]},
        {"plot_vars": ["Rs"]},
        {"plot_vars": ["Rp"]},
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
        num_periods=num_periods,
        theta=theta,
        include=["LOPC"],
        label_override="",
    )
    for theta in [0, 15, 30, 45]
}

In [ ]:
grid = hv.GridSpace(plots, kdims=["θ"])*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Overlay(show_legend=False, show_grid=True))
grid[0].opts(opts.Overlay(show_legend=True, legend_position="left"))

grid.opts(opts.Curve(xticks=2, yticks=3, frame_height=120, frame_width=120), opts.VLine(line_dash="dotted", color=yellow), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
# hv.save(grid, filename=fig_path/"R_lambda_theta_1D", fmt="png", toolbar=None)

In [ ]:
# Brewster's angle
np.arctan(1.45/1.35)*(180/np.pi)

In [ ]:
plots = OrderedDict({
    var: plot_R(
        variable=var,
        dataset=restacked_ds.sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
            num_periods=num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["R", "Rs", "Rp"]
})

In [ ]:
grid = hv.GridSpace(plots, kdims=["Reflectance"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Image(colorbar=False))
grid["R"].opts(opts.Image(colorbar=True, colorbar_position="left"))

grid.opts(opts.Image(xticks=3, frame_height=120, frame_width=160), opts.VLine(line_dash="dotted", color="white"), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
# hv.save(grid, filename=fig_path/"R_lambda_theta_2D", fmt="png", toolbar=None)

## Other high-angle stuff

AEF curving with theta

In [ ]:
period = 300
excitonic_layer_thickness = 50
passive_layer_thickness = period - excitonic_layer_thickness
num_periods = 20

In [ ]:
plots = OrderedDict({
    var: plot_ef(
        variable=var,
        dataset=restacked_norm_1.sel(
            period=period,
            excitonic_layer_thickness=excitonic_layer_thickness,
            num_periods=num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["A", "As", "Ap"]
})

In [ ]:
grid = hv.GridSpace(plots, kdims=["γA"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
grid.opts(opts.Image(colorbar=False))
grid["A"].opts(opts.Image(colorbar=True, colorbar_position="left"))

grid.opts(opts.Image(clim=(0, 2), xticks=3, frame_height=120, frame_width=160), opts.VLine(line_dash="dotted", color="black"), opts.GridSpace(fontsize={"xlabel": 8}, toolbar="below"))

In [ ]:
hv.save(grid, filename=fig_path/"Aef_lambda_theta_2D", fmt="png", toolbar=None)

In [ ]:
layout = hv.NdLayout(plots, kdims=["γA"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
layout["A"].opts(opts.Image(colorbar=True, colorbar_position="left"))
layout["As"].opts(opts.Image(colorbar=False, yaxis="bare"))
layout["Ap"].opts(opts.Image(colorbar=False, yaxis="bare"))

layout.opts(opts.Image(clim=(0, 2), xticks=3, frame_height=120, frame_width=160), opts.VLine(line_dash="dotted", color="black"))

In [ ]:
hv.save(layout["A"], filename=fig_path/"Aef_lambda_theta_2D", fmt="png", toolbar=None)
hv.save(layout["As"], filename=fig_path/"Asef_lambda_theta_2D", fmt="png", toolbar=None)
hv.save(layout["Ap"], filename=fig_path/"Apef_lambda_theta_2D", fmt="png", toolbar=None)

In [ ]:
# maths to work out float sizes in LaTeX
print(0.479768786*0.95)
print(0.2601156069*0.95)

In [ ]:
plots = OrderedDict({
    var: plot_vars_to_funcs(var)[0](
        variable=var,
        dataset=crs_1.sel(
            total_excitonic_thickness=excitonic_layer_thickness*num_periods,
        ).squeeze(),
        x="wavelength",
        y="theta",
    )
    for var in ["R", "T", "A"]
})

In [ ]:
layout = hv.NdLayout(plots, kdims=["FOM"], sort=False)*lorentz_vlines(0, scale=1e-9)

In [ ]:
layout["R"].opts(opts.VLine(color="white"))
layout["T"].opts(opts.VLine(color="black"), opts.Image(yaxis="bare"))
layout["A"].opts(opts.VLine(color="black"), opts.Image(yaxis="bare"))

layout.opts(opts.Image(xticks=3, frame_height=120, frame_width=140), opts.VLine(line_dash="dotted"))

In [ ]:
hv.save(layout["R"], filename=fig_path/"R_CRS_lambda_theta_2D", fmt="png", toolbar=None)
hv.save(layout["T"], filename=fig_path/"T_CRS_lambda_theta_2D", fmt="png", toolbar=None)
hv.save(layout["A"], filename=fig_path/"A_CRS_lambda_theta_2D", fmt="png", toolbar=None)

In [ ]:
# maths to work out float sizes in LaTeX
total_pixel_width = 302 + 237 + 237
print(302/total_pixel_width*0.96)
print(237/total_pixel_width*0.96)

## Field profiles around PBG

In [ ]:
field_curves = {
    ("dummy", wl): wrap_plot_field(
        wavelength=wl,
        excitonic_layer_thickness=excitonic_layer_thickness,
        passive_layer_thickness=passive_layer_thickness,
        num_periods=num_periods,
        theta=0,
        z_scale=1e-6,
    ).Curve
    for wl in [785, 825, 875]
}

In [ ]:
structure_vspans = wrap_plot_field(
    wavelength=785,
    excitonic_layer_thickness=excitonic_layer_thickness,
    passive_layer_thickness=passive_layer_thickness,
    num_periods=num_periods,
    z_scale=1e-6
).VSpan

In [ ]:
# # a legend on a single Curve is impossible
# hv.Curve(([0,1,2],[2,1,0]), label="foo").opts(opts.Curve(show_legend=True))

In [ ]:
grid = (hv.GridSpace(field_curves, kdims=["dummy", wavelength_dim]) * structure_vspans).redim(z=z_dim, E_squared=E_dim).opts(
    opts.Curve(xticks=6, yticks=4, frame_width=600, frame_height=100, padding=(0.1, 0), ylim=(-0.1, 1.8)),
    opts.GridSpace(xaxis=None, toolbar="below"),
)
grid

In [ ]:
hv.save(grid, filename=fig_path/"LOPC_Efield_example", fmt="png", toolbar=None)

## N-Lambda grid plot

This grid plot demonstrates the shift and growth in PBG with period and N.

In [ ]:
excitonic_layer_thickness = 30
theta = 0

In [ ]:
coords = {"excitonic_layer_thickness": excitonic_layer_thickness, "theta": theta}

In [ ]:
def label_subplot():
    try:
        label_subplot.i += 1
    except AttributeError:
        label_subplot.i = 0
    label = string.ascii_lowercase[label_subplot.i]
    return f"({label})"

In [ ]:
curves = {
    (P, N): restacked_ds[unpolarised_RTA]
    .sel(num_periods=N, period=P, **coords)
    .squeeze()
    .hvplot.line(x="wavelength")
    * lorentz_vlines(0, scale=1e-9).opts(line_color=yellow)
    * hv.VLine(pbg_position(excitonic_layer_thickness, P, n1=1.5, n2=1.35)).opts(
        line_color=blue
    )
    * hv.Labels((510, 0.7, label_subplot())).opts(text_color="black")
    for N in [50, 20, 10]
    for P in [200, 250, 300]
}

In [ ]:
grid = hv.GridSpace(curves, kdims=[period_dim, num_periods_dim])

In [ ]:
# hacky workaround to bugs in hvplot/holoviews
for p in grid:
    p.opts(
        opts.Overlay(
            # show_legend=False,
            show_grid=True,
        ),
        opts.Curve(
            color=hv.Cycle([blue, yellow, red]),  # set R/T/A->blue/yellow/red
            frame_width=200,
        ),
    )  # remove the excess legends

# for p in grid[250] or grid[300]:
#     p.opts(
#             opts.Overlay(
#                 show_legend=False,
#                 show_grid=True,
#             ),
#             opts.Curve(
#                 color=hv.Cycle([blue, yellow, red]),  # set R/T/A->blue/yellow/red
#                 frame_width=200,
#             ),
#         )  # remove the excess legends

# grid[200, 50].opts(
#     opts.Overlay(show_legend=True, legend_position="left")
# )  # add the legend back in on a free side but this breaks the left alignment somehow

grid.opts(
    opts.Curve(ylim=(0,1), toolbar=None, hooks=[outline_hook]),
    opts.Overlay(fontscale=1.5),
    opts.VLine(line_dash="dotted", line_width=2),
    opts.GridSpace(
        show_legend=False, shared_xaxis=True, shared_yaxis=True, toolbar=None, fontscale=1.5
    ),
)

This is the code I would have liked to have used to save this image.

Unfortunately the `toolbar=False` argument doesn't work so I have to actually *use* the toolbar above to save it!

... Except this ALSO doesn't work, because the save button saves 9 individual images?! So I guess I have to use the below code and manually crop or something stupid.

In [ ]:
hv.save(grid, filename=fig_path/f"N_Lambda_grid_2", fmt="png", toolbar=None)

## N-n grid plot

This grid plot demonstrates the shift and growth in PBG with n_p and N.

In [ ]:
excitonic_layer_thickness = 50
period = 200
theta = 0

In [ ]:
coords = {"excitonic_layer_thickness": excitonic_layer_thickness, "theta": theta, "period": period}

In [ ]:
def label_subplot():
    try:
        label_subplot.i += 1
    except AttributeError:
        label_subplot.i = 0
    label = string.ascii_lowercase[label_subplot.i]
    return f"({label})"

In [ ]:
curves = {
    (R, N): restacked_np_ds[unpolarised_RTA]
    .sel(num_periods=N, real_RI=R, **coords)
    .squeeze()
    .hvplot.line(x="wavelength")
    * lorentz_vlines(0, scale=1e-9).opts(line_color=yellow)
    * hv.VLine(pbg_position(excitonic_layer_thickness, period, n1=1.5, n2=R)).opts(
        line_color=blue
    )
    * hv.Labels((850, 0.7, label_subplot())).opts(text_color="black")
    for N in [50, 20, 10]
    for R in [1.3, 1.35, 1.4]
}

In [ ]:
grid = hv.GridSpace(curves, kdims=[real_index_dim, num_periods_dim]).redim(n="n_p")

In [ ]:
# hacky workaround to bugs in hvplot/holoviews
for p in grid:
    p.opts(
        opts.Overlay(
            # show_legend=False,
            show_grid=True,
        ),
        opts.Curve(
            color=hv.Cycle([blue, yellow, red]),  # set R/T/A->blue/yellow/red
            frame_width=200,
        ),
    )  # remove the excess legends

# grid[200, 50].opts(
#     opts.Overlay(show_legend=True, legend_position="left")
# )  # add the legend back in on a free side but this breaks the left alignment somehow

grid.opts(
    opts.Curve(ylim=(0,1), toolbar=None, hooks=[outline_hook]),
    opts.Overlay(fontscale=1.5),
    opts.VLine(line_dash="dotted", line_width=2),
    opts.GridSpace(
        show_legend=False, shared_xaxis=True, shared_yaxis=True, toolbar=None, fontscale=1.5
    ),
)

This is the code I would have liked to have used to save this image.

Unfortunately the `toolbar=False` argument doesn't work so I have to actually *use* the toolbar above to save it!

... Except this ALSO doesn't work, because the save button saves 9 individual images?! So I guess I have to use the below code and manually crop or something stupid.

In [ ]:
hv.save(grid.select(period=200), filename=fig_path/f"N_n_grid_2", fmt="png", toolbar=False)

## 2D plots vs period

In [ ]:
theta = 0
num_periods = 10
excitonic_layer_thickness = 30
period = 200

In [ ]:
period_slice = hv.HLine(period)

In [ ]:
temp_params = {
    "theta": theta,
    "num_periods": num_periods,
    "excitonic_layer_thickness": excitonic_layer_thickness,
}

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide by zero")
    warnings.filterwarnings("ignore", "invalid value encountered in divide")
    the_plot = plot_R(variable="R", dataset=restacked_ds.sel(**temp_params).dropna("period").squeeze(), x="wavelength", y="period")

In [ ]:
the_plot = (
    (the_plot * lorentz_vlines(0, scale=1e-9)*period_slice)
    .redim(period=period_dim)
    .opts(
        opts.VLine(line_dash="dotted", color="white"),
        opts.HLine(line_dash="dotted", color="white"),
        opts.Image(clim=(0, None), title=""),
    )
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_LOPC_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
# the_plot = (
#     plot_RTA(
#         period=period,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#         num_periods=num_periods,
#         theta=theta,
#         plot_vars=["R"],
#         include=["LOPC", "CRS_1", "FRS_1"],
#     )
#     * lorentz_vlines(0, scale=1e-9)
# ).opts(
#     opts.VLine(line_dash="dotted", color=yellow),
#     opts.Curve(ylim=(0, None)),
# )
# display(the_plot)

In [ ]:
# hv.save(the_plot, filename=fig_path/f"R_comparison_1D_wl_P{period}_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
# the_plot = (
#     plot_RTA(
#         period=period,
#         excitonic_layer_thickness=excitonic_layer_thickness,
#         num_periods=num_periods,
#         theta=theta,
#         plot_vars=["A"],
#         include=["LOPC", "CRS_1", "FRS_1"],
#     )
#     * lorentz_vlines(0, scale=1e-9)
# ).opts(opts.VLine(line_dash="dotted", color=yellow))
# display(the_plot)

In [ ]:
# hv.save(the_plot, filename=fig_path/f"A_comparison_1D_wl_P{period}_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
the_plot = (
    plot_RTA(
        period=period,
        excitonic_layer_thickness=excitonic_layer_thickness,
        num_periods=num_periods,
        theta=theta,
        plot_vars=["R", "A"],
        include=["LOPC", "CRS_1", "FRS_1"],
    )
    * lorentz_vlines(0, scale=1e-9)
).opts(
    opts.VLine(line_dash="dotted", color=yellow),
    opts.Overlay(legend_position="right"),
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"RA_comparison_1D_wl_P{period}_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
the_plot = plot_ef(variable="A", dataset=restacked_norm_1.sel(**temp_params).dropna("period").squeeze(), x="wavelength", y="period")

In [ ]:
the_plot = (
    (the_plot * lorentz_vlines(0, scale=1e-9) * period_slice)
    .redim(period=period_dim)
    .opts(
        opts.VLine(line_dash="dotted", color="black"),
        opts.HLine(line_dash="dotted", color="black"),
        opts.Image(title=""),
    )
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"AEF_CRS_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
the_plot = plot_ef(variable="A", dataset=restacked_norm_2.sel(**temp_params).dropna("period").squeeze(), x="wavelength", y="period")

In [ ]:
the_plot = (
    (the_plot * lorentz_vlines(0, scale=1e-9) * period_slice)
    .redim(period=period_dim)
    .opts(
        opts.VLine(line_dash="dotted", color="black"),
        opts.HLine(line_dash="dotted", color="black"),
        opts.Image(cmap="Blues_r", clim=(0,1), title=""),
    )
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"AEF_FRS_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide by zero")
    warnings.filterwarnings("ignore", "invalid value encountered in divide")
    the_plot = plot_df(variable="A", dataset=restacked_diff_1.sel(**temp_params).dropna("period").squeeze(), x="wavelength", y="period")

In [ ]:
the_plot = (
    (the_plot * lorentz_vlines(0, scale=1e-9) * period_slice)
    .redim(period=period_dim)
    .opts(
        opts.VLine(line_dash="dotted", color="black"),
        opts.HLine(line_dash="dotted", color="black"),
        opts.Image(clim=(-0.25, 0.25), title=""),
    )
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"ADF_CRS_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide by zero")
    warnings.filterwarnings("ignore", "invalid value encountered in divide")
    the_plot = plot_df(variable="A", dataset=restacked_diff_2.sel(**temp_params).squeeze(), x="wavelength", y="period")  #drop_na("period") takes forever

In [ ]:
the_plot = (
    (the_plot * lorentz_vlines(0, scale=1e-9) * period_slice)
    .redim(period=period_dim)
    .opts(
        opts.VLine(line_dash="dotted", color="black"),
        opts.HLine(line_dash="dotted", color="black"),
        opts.Image(ylim=(25, 345), cmap="Blues_r", clim=(-0.8, 0), title=""),
    )
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"ADF_FRS_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
ef_plots = [plot_ef(variable="A", dataset=dataset, sel={
        "period": period,
        "num_periods": num_periods,
        "excitonic_layer_thickness": excitonic_layer_thickness,
        "theta": theta,
    }).relabel(label, depth=1) for dataset, label in zip([restacked_norm_1, restacked_norm_2], ["CRS", "FRS"])]

In [ ]:
the_plot = (hv.Overlay(ef_plots) * lorentz_vlines(0, scale=1e-9)).opts(
    opts.Curve(ylabel="γA", line_dash=default_dash_cycle),
    opts.VLine(line_dash="dotted", color=yellow),
    opts.HLine(line_dash="dotted", color=blue),
    opts.Overlay(legend_position="bottom_right"),
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"AEF_comparison_2D_wl_P{period}_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
df_plots = [plot_df(variable="A", dataset=dataset, sel={
        "period": period,
        "num_periods": num_periods,
        "excitonic_layer_thickness": excitonic_layer_thickness,
        "theta": theta,
    }).relabel(label, depth=1) for dataset, label in zip([restacked_diff_1, restacked_diff_2], ["CRS", "FRS"])]

In [ ]:
the_plot = (hv.Overlay(df_plots) * lorentz_vlines(0, scale=1e-9)).opts(
    opts.Curve(ylabel="ΔA", line_dash=default_dash_cycle),
    opts.VLine(line_dash="dotted", color=yellow),
    opts.HLine(line_dash="dotted", color=blue),
    opts.Overlay(legend_position="bottom_right"),
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"ADF_comparison_1D_wl_P{period}_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

## Integrated absorptance enhancement

In [ ]:
def integrated_cross_section(
    dataset,
    variable=None,
    wavelength=None,
    integrate_wavelength=None,
    theta=None,
    integrate_theta=None,
    normalisation=1,
):
    da = dataset[variable] if variable is not None else dataset
    integration_dims = []

    match (wavelength, integrate_wavelength):
        case None, None:
            pass
        case None, _:
            raise ValueError("'integrate_wavelength' supplied but not 'wavelength'")
        case _, None:
            da = da.sel(wavelength=wavelength, method="nearest")
        case _, _:
            da = da.sel(wavelength=slice(wavelength, integrate_wavelength))
            integration_dims.append("wavelength")

    match (theta, integrate_theta):
        case None, None:
            pass
        case None, _:
            raise ValueError("'integrate_theta' supplied but not 'theta'")
        case _, None:
            da = da.sel(theta=theta, method="nearest")
        case _, _:
            da = da.sel(theta=slice(theta, integrate_theta))
            integration_dims.append("theta")

    if integration_dims:  # if the list isn't empty
        da = integrate_da(
            da, integration_dims, weighting=1, normalisation=normalisation
        )

    return da

In [ ]:
def integrated_enhancement_factor(
    ds,
    ref,
    common_dim,
    variable=None,
    wavelength=None,
    integrate_wavelength=None,
    theta=None,
    integrate_theta=None,
    method="groupby",
    plt_to_period=False,
):
    ds = integrated_cross_section(
        ds,
        variable=variable,
        wavelength=wavelength,
        integrate_wavelength=integrate_wavelength,
        theta=theta,
        integrate_theta=integrate_theta,
        normalisation=None,
    )
    ref = integrated_cross_section(
        ref,
        variable=variable,
        wavelength=wavelength,
        integrate_wavelength=integrate_wavelength,
        theta=theta,
        integrate_theta=integrate_theta,
        normalisation=None,
    )
    
    ef = enhancement_factor(ds=ds, ref=ref, common_dim=common_dim, method=method)
    
    if plt_to_period:
        ef = restack_plt_to_period(ef)
    
    return ef

In [ ]:
theta = 0
num_periods = 10
excitonic_layer_thickness = 30
period = 200

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "divide by zero")
#     warnings.filterwarnings("ignore", "invalid value")
#     the_plot = plot_ef(
#         "A",
#         integrated_enhancement_factor(
#             ds,
#             crs_1,
#             "total_excitonic_thickness",
#             None,
#             wavelength=680,
#             integrate_wavelength=None,
#             theta=theta,
#             integrate_theta=None,
#             plt_to_period=True,
#         ).squeeze(),
#         sel=dict(
#             num_periods=num_periods, excitonic_layer_thickness=excitonic_layer_thickness
#         ),
#         x="period",
#     )
# display(the_plot)

In [ ]:
# with warnings.catch_warnings():
#     warnings.filterwarnings("ignore", "divide by zero")
#     warnings.filterwarnings("ignore", "invalid value")
#     the_plot = (
#         integrated_enhancement_factor(
#             ds,
#             crs_1,
#             "total_excitonic_thickness",
#             "A",
#             wavelength=680,
#             integrate_wavelength=None,
#             theta=theta,
#             integrate_theta=None,
#             plt_to_period=True,
#         )
#         .sel(period=slice(30, 340))
#         .sel(excitonic_layer_thickness=30)
#         .squeeze()
#         .hvplot(kind="quadmesh", x="period", y="num_periods")
#     )
#     display(the_plot)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide by zero")
    warnings.filterwarnings("ignore", "invalid value")
    ef_plots = [
        plot_ef(
            "A",
            integrated_enhancement_factor(
                ds,
                crs_1,
                "total_excitonic_thickness",
                None,
                wavelength=ds.wavelength.min(),
                integrate_wavelength=ds.wavelength.max(),
                theta=theta,
                integrate_theta=integrate_theta,
                plt_to_period=True,
            ).squeeze(),
            sel=dict(
                num_periods=num_periods,
                excitonic_layer_thickness=excitonic_layer_thickness,
            ),
            x="period",
        )
        .relabel(label, depth=1)
        .opts(opts.Curve(line_dash=line_dash))
        for integrate_theta, label, line_dash in zip(
            [None, 45], ["normal incidence", "integrated"], ["solid", "dashed"]
        )
    ]
hv.Overlay(ef_plots)

In [ ]:
the_plot = (hv.Overlay(ef_plots)).redim(period=period_dim).opts(
    opts.Curve(ylabel="γA_tot", xlim=(None, 350)),
    opts.HLine(line_dash="dotted", color=blue),
    opts.Overlay(legend_position="bottom_left"),
)
display(the_plot)

In [ ]:
hv.save(the_plot, filename=fig_path/f"TAEF_comparison_1D_P_th_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}-45", fmt="png", toolbar=None)

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", "divide by zero")
    warnings.filterwarnings("ignore", "invalid value")
    ef_plots = [
        plot_ef(
            "A",
            integrated_enhancement_factor(
                ds,
                crs_1,
                "total_excitonic_thickness",
                None,
                wavelength=ds.wavelength.min(),
                integrate_wavelength=ds.wavelength.max(),
                theta=theta,
                integrate_theta=integrate_theta,
                plt_to_period=True,
            ).squeeze(),
            sel=dict(
                num_periods=num_periods,
                excitonic_layer_thickness=excitonic_layer_thickness,
            ),
            x="period",
        )
        .relabel(label, depth=1)
        .opts(opts.Curve(line_dash=line_dash))
        for theta, integrate_theta, label, line_dash in zip(
            [0, 15, 30, 0], [None, None, None, 45], ["normal incidence", "15", "30", "integrated"], ["solid", "dashed", "dotted", "dashdot"]
        )
    ]
hv.Overlay(ef_plots)

## Total absorptance enhancement factor

Plot total absorption enhancement factor under an arbitrary spectrum.

##### Plot total absorptance enhancement factor as a function of period.

In [ ]:
ymin1 = float(ds_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).min())
ymax1 = float(ds_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).max())

In [ ]:
ymin2 = float(norm_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).min())
ymax2 = float(norm_flat_spectrum.A.drop_sel(excitonic_layer_thickness=0).sel(theta=0).max())

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(x="passive_layer_thickness", y="excitonic_layer_thickness", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
(
    restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0)
    .squeeze()
    .A.hvplot(x="period", y="excitonic_layer_thickness", clim=(0.8, 1.2), cmap="RdBu_r")
    .opts(clabel="γ_tot (flat spectrum)", shared_axes=False)
    + restacked_ds_flat_spectrum.drop_sel(excitonic_layer_thickness=0)
    .squeeze()
    .A.hvplot(x="period", y="excitonic_layer_thickness", clim=(0, 1), cmap="inferno")
    .opts(clabel="A_tot", shared_axes=False)
).cols(1)

##### Define some convenient datasets for plotting.

In [ ]:
crs_1_fs_like_ds = crs_1_flat_spectrum.sel(
    total_excitonic_thickness=ds.total_excitonic_thickness
).assign(passive_layer_thickness = ds_flat_spectrum.passive_layer_thickness)

In [ ]:
restacked_crs_1_fs_like_ds = (
    assign_derived_attrs(
        crs_1_fs_like_ds,
        absorption=False,
        unpolarised=False,
        period=True,
        total_excitonic_thickness=False,
        total_passive_thickness=False,
        total_thickness=False,
        N_tot=False,
        per_oscillator=False,
    )
    .stack(multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"])
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

In [ ]:
diff_1_flat_spectrum = ds_flat_spectrum - crs_1_fs_like_ds

In [ ]:
restacked_diff_1_flat_spectrum = (
    diff_1_flat_spectrum.stack(
        multiperiod=["passive_layer_thickness", "excitonic_layer_thickness"]
    )
    .set_index(multiperiod=["period", "excitonic_layer_thickness"])
    .unstack()
)

##### Plot the total absorptance and absolute difference in total absorptance

In [ ]:
restacked_ds_flat_spectrum[unpolarised_RTA].squeeze().hvplot(
    x="period", ylim=(0, 1)
)

In [ ]:
crs_1[unpolarised_RTA].squeeze().hvplot(x="wavelength", ylim=(0,1))*lorentz_vlines(0, scale=1e-9).opts(line_dash='dotted', color=yellow)

In [ ]:
(
    crs_1_flat_spectrum[unpolarised_RTA]
    .squeeze()
    .hvplot(
        x="total_excitonic_thickness",
        xlim=(0, 3500),
        ylim=(0, 1),
    )
    * hv.VLine(120, label="6 periods of 20 nm").opts(line_dash="dotted")
    * hv.VLine(3000, label="50 periods of 60 nm").opts(line_dash="dotted")
)

In [ ]:
num_periods = 6
excitonic_layer_thickness = 20
theta = 0
periods = [40, 160, 210, 260]

In [ ]:
dmap = (restacked_diff_1_flat_spectrum)[unpolarised_RTA].squeeze().hvplot(
    x="period",
)*hv.HLine(0).opts(line_dash='dotted')
dmap

In [ ]:
the_plot = (
    dmap[num_periods, theta, excitonic_layer_thickness] * vlines(periods)
).opts(
    opts.VLine(line_dash="dotted"),
    opts.Curve(xlabel="Λ (nm)", xlim=(None, 330), color=hv.Cycle([blue, yellow, red])),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"opt_Aef_max_0_5_0_0_RTA_df_0", fmt="png", toolbar=None)

In [ ]:
# manually saved opt_Aef_max_0_5_0_0_RTA_df_0.png at N=6, d_e=20

In [ ]:
# (integrate_da(restacked_diff_1.sel(wavelength=slice(580, 780)), dim="wavelength", normalisation=1))[unpolarised_RTA].squeeze().hvplot(
#     x="period",
# )*hv.HLine(0).opts(line_dash='dotted')

In [ ]:
# (integrate_da(restacked_diff_1.sel(wavelength=slice(630, 730)), dim="wavelength", normalisation=1))[unpolarised_RTA].squeeze().hvplot(
#     x="period",
# )*hv.HLine(0).opts(line_dash='dotted')

##### Plot the AEF at periods to illustrate what's going on above

In [ ]:
the_plot = (
    restacked_norm_1["A"]
    .squeeze()
    .sel(
        excitonic_layer_thickness=excitonic_layer_thickness,
        num_periods=num_periods,
        theta=theta,
    )
    .sel(period=periods)
    .hvplot(x="wavelength", by="period")
    * hv.HLine(1)
    * lorentz_vlines(0, scale=1e-9)
).opts(
    opts.HLine(line_dash="dotted"),
    opts.VLine(line_dash="dotted", color=yellow),
    opts.Curve(ylabel="γA", line_dash=hv.Cycle(["dashdot", "dotdash", "solid", "dashed"])),
    opts.Overlay(legend_opts = {"title": "Λ"}),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/f"AEF_comparison_1D_wl_P110-260_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

##### Plot total absorption enhancement factor as a function of excitonic layer thickness and number of layers.

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

##### Plot total absorption enhancement factor as a function of passive layer thickness/period and number of layers.

In [ ]:
norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="passive_layer_thickness", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

In [ ]:
restacked_norm_flat_spectrum.drop_sel(excitonic_layer_thickness=0).squeeze().A.hvplot(kind="quadmesh", x="period", y="num_periods", clim=(0.8, 1.2), cmap='RdBu_r')

### Optimisation of total absorption FOMs

##### Find maximum gamma_tot

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(None, None), 'theta': slice(0,0)}
da = restacked_norm_flat_spectrum['A'].drop_sel(excitonic_layer_thickness=0).sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
plot_RTA(
    period=P,
    excitonic_layer_thickness=t,
    num_periods=N,
    theta=0,
)

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash="dashed"))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash="dashed"))
    # * resonance_line
).opts(
    ylim=(0, 1),
    ylabel="Intensity",
    title=f"period: {P} nm\nexcitonic_layer_thickness: {t} nm\nnum_periods: {N}",
)

##### Find maximum Delta_tot

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(None,None), 'theta': slice(0,0)}
da = restacked_diff_1_flat_spectrum['A'].sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
plot_RTA(
    period=P,
    excitonic_layer_thickness=t,
    num_periods=N,
    theta=0,
)

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash='dashed'))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash='dashed'))
    *resonance_line
).opts(
    ylim=(0, 1),
    ylabel="Intensity",
    title=f"period: {P} nm\nexcitonic_layer_thickness: {t} nm\nnum_periods: {N}",
)

##### Find maximum... Delta_absorption_per_oscillator_tot?

In [ ]:
sel = {'excitonic_layer_thickness': slice(None,None), 'period': slice(None,None), 'num_periods': slice(None, None), 'theta': slice(0,0)}
da = restacked_diff_1_flat_spectrum['A_per_oscillator'].drop_sel(excitonic_layer_thickness=0).sel(sel)
optimum = da.where(da==da.max(), drop=True)
print(optimum.values)
print(optimum.coords)
P = optimum.period.values[0]
t = optimum.excitonic_layer_thickness.values[0]
N = optimum.num_periods.values[0]

In [ ]:
(
    restacked_ds["R"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (LOPC)")
    * restacked_ds["A"]
    .sel(period=P, excitonic_layer_thickness=t, num_periods=N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (LOPC)")
    * crs_1["R"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Reflectance (CRS)")
    .opts(opts.Curve(color=blue, line_dash='dashed'))
    * crs_1["A"]
    .sel(total_excitonic_thickness=t * N, theta=0)
    .squeeze()
    .hvplot(x="wavelength", label="Absorptance (CRS)")
    .opts(opts.Curve(color=red, line_dash='dashed'))
    *resonance_line
).opts(ylim=(0, 1), ylabel="Intensity")

## Tables of optimal parameters

In [ ]:
variable = pd.CategoricalDtype(['R', 'T', 'A', 'R_per_oscillator', 'T_per_oscillator', 'A_per_oscillator'], 
                         ordered=True)

In [ ]:
# # old caption recorded in case I want to use it after all
# caption=(
#         "Table of optimal parameters for \gls{lopc} \glspl{fom}, indexed by: the \gls{fom} in question;"
#         " the type of optimisation (maximum or minimum); the wavelength of interest (denoted by the"
#         " Lorentz line index) and wavelength window radius in Lorentz linewidths; the angle of incidence"
#         " \gls{theta}, and the angular integration limit. Where the integration limit is recorded as 0,"
#         " it represents no angular integral, e.g.\ \(\gls{theta}=30\), \gls{theta} integration limit = 0"
#         " indicates the optimisation was at \SI{30}{\degree}.",
#         "Table of optimal parameters for \gls{lopc} \glspl{fom}",
#     ),

In [ ]:
opt_LOPC = pd.read_csv(
    data_path / f"run_{run_number}/optimal_LOPC.csv",
    # index_col=[
    #     "variable",
    #     "maxmin",
    #     "lorentz_line",
    #     "window_radius",
    #     "theta_start",
    #     "theta_stop",
    # ],
)

opt_LOPC["variable"] = opt_LOPC["variable"].astype(variable)

opt_LOPC = opt_LOPC.sort_values(
    by=["variable", "maxmin", "lorentz_line", "window_radius"]
)

opt_LOPC = opt_LOPC.rename(
    columns={
        "variable": "\gls{fom}",
        "maxmin": "Max/min",
        "lorentz_line": "Lorentz line",
        "window_radius": "Window radius",
        "theta_start": "\gls{theta}",
        "theta_stop": "$\gls{theta}_i$",
        "excitonic_layer_thickness": "\gls{d_e}",
        "num_periods": "\gls{N_lopc}",
        "period": "\gls{Lambda}",
        "value": "Value"
    }
)

opt_LOPC = opt_LOPC.replace(
    {
        "R": "\gls{R}",
        "T": "\gls{T}",
        "A": "\gls{A}",
        "R_per_oscillator": "$\po{\gls{R}}$",
        "T_per_oscillator": "$\po{\gls{T}}$",
        "A_per_oscillator": "$\po{\gls{A}}$",
        "max": "Max",
        "min": "Min",
    }
)

opt_LOPC = opt_LOPC.set_index(list(opt_LOPC.columns[0:6]))

opt_LOPC.drop(columns=["period_start", "period_stop"]).to_latex(
    buf=root / r"thesis\LaTeX\chapters\appendices\optimal_LOPC_parameters.tex",
    float_format="{:.3e}".format,
    sparsify=True,
    longtable=True,
    escape=False,
    label="tab:optimal_LOPC_parameters",
    position="ht",
    caption="Table of optimal parameters for \gls{lopc} \glspl{fom}",
)

In [ ]:
opt_diff_1 = pd.read_csv(
    data_path / f"run_{run_number}/optimal_diff_1.csv",
    # index_col=[
    #     "variable",
    #     "maxmin",
    #     "lorentz_line",
    #     "window_radius",
    #     "theta_start",
    #     "theta_stop",
    # ],
)

opt_diff_1["variable"] = opt_diff_1["variable"].astype(variable)

opt_diff_1 = opt_diff_1.sort_values(
    by=["variable", "maxmin", "lorentz_line", "window_radius"]
)

opt_diff_1 = opt_diff_1.rename(
    columns={
        "variable": "\gls{fom}",
        "maxmin": "Max/min",
        "lorentz_line": "Lorentz line",
        "window_radius": "Window radius",
        "theta_start": "\gls{theta}",
        "theta_stop": "$\gls{theta}_i$",
        "excitonic_layer_thickness": "\gls{d_e}",
        "num_periods": "\gls{N_lopc}",
        "period": "\gls{Lambda}",
        "value": "Value",
    }
)

opt_diff_1 = opt_diff_1.replace(
    {
        "R": "$\df{\gls{R}, \mathrm{\gls{crs}}}$",
        "T": "$\df{\gls{T}, \mathrm{\gls{crs}}}$",
        "A": "$\df{\gls{A}, \mathrm{\gls{crs}}}$",
        "R_per_oscillator": "$\df{\po{\gls{R}}, \mathrm{\gls{crs}}}$",
        "T_per_oscillator": "$\df{\po{\gls{T}}, \mathrm{\gls{crs}}}$",
        "A_per_oscillator": "$\df{\po{\gls{A}}, \mathrm{\gls{crs}}}$",
        "max": "Max",
        "min": "Min",
    }
)

opt_diff_1 = opt_diff_1.set_index(list(opt_diff_1.columns[0:6]))

opt_diff_1.drop(columns=["period_start", "period_stop"]).to_latex(
    buf=root / r"thesis\LaTeX\chapters\appendices\optimal_diff_1_parameters.tex",
    float_format="{:.3e}".format,
    sparsify=True,
    longtable=True,
    escape=False,
    label="tab:optimal_diff_1_parameters",
    position="ht",
    caption="Table of optimal parameters for \gls{lopc} \gls{fom} difference factors relative to \gls{crs}",
)

In [ ]:
opt_diff_2 = pd.read_csv(
    data_path / f"run_{run_number}/optimal_diff_2.csv",
    # index_col=[
    #     "variable",
    #     "maxmin",
    #     "lorentz_line",
    #     "window_radius",
    #     "theta_start",
    #     "theta_stop",
    # ],
)

opt_diff_2["variable"] = opt_diff_2["variable"].astype(variable)

opt_diff_2 = opt_diff_2.sort_values(
    by=["variable", "maxmin", "lorentz_line", "window_radius"]
)

opt_diff_2 = opt_diff_2.rename(
    columns={
        "variable": "\gls{fom}",
        "maxmin": "Max/min",
        "lorentz_line": "Lorentz line",
        "window_radius": "Window radius",
        "theta_start": "\gls{theta}",
        "theta_stop": "$\gls{theta}_i$",
        "excitonic_layer_thickness": "\gls{d_e}",
        "num_periods": "\gls{N_lopc}",
        "period": "\gls{Lambda}",
        "value": "Value",
    }
)

opt_diff_2 = opt_diff_2.replace(
    {
        "R": "$\df{\gls{R}, \mathrm{\gls{frs}}}$",
        "T": "$\df{\gls{T}, \mathrm{\gls{frs}}}$",
        "A": "$\df{\gls{A}, \mathrm{\gls{frs}}}$",
        "R_per_oscillator": "$\df{\po{\gls{R}}, \mathrm{\gls{frs}}}$",
        "T_per_oscillator": "$\df{\po{\gls{T}}, \mathrm{\gls{frs}}}$",
        "A_per_oscillator": "$\df{\po{\gls{A}}, \mathrm{\gls{frs}}}$",
        "max": "Max",
        "min": "Min",
    }
)

opt_diff_2 = opt_diff_2.set_index(list(opt_diff_2.columns[0:6]))

opt_diff_2.drop(columns=["period_start", "period_stop"]).to_latex(
    buf=root / r"thesis\LaTeX\chapters\appendices\optimal_diff_2_parameters.tex",
    float_format="{:.3e}".format,
    sparsify=True,
    longtable=True,
    escape=False,
    label="tab:optimal_diff_2_parameters",
    position="ht",
    caption="Table of optimal parameters for \gls{lopc} \gls{fom} difference factors relative to \gls{frs}",
)

In [ ]:
opt_norm_1 = pd.read_csv(
    data_path / f"run_{run_number}/optimal_norm_1.csv",
    # index_col=[
    #     "variable",
    #     "maxmin",
    #     "lorentz_line",
    #     "window_radius",
    #     "theta_start",
    #     "theta_stop",
    # ],
)

opt_norm_1["variable"] = opt_norm_1["variable"].astype(variable)

opt_norm_1 = opt_norm_1.sort_values(
    by=["variable", "maxmin", "lorentz_line", "window_radius"]
)

opt_norm_1 = opt_norm_1.rename(
    columns={
        "variable": "\gls{fom}",
        "maxmin": "Max/min",
        "lorentz_line": "Lorentz line",
        "window_radius": "Window radius",
        "theta_start": "\gls{theta}",
        "theta_stop": "$\gls{theta}_i$",
        "excitonic_layer_thickness": "\gls{d_e}",
        "num_periods": "\gls{N_lopc}",
        "period": "\gls{Lambda}",
        "value": "Value",
    }
)

opt_norm_1 = opt_norm_1.replace(
    {
        "R": "$\ef{\gls{R}, \mathrm{\gls{crs}}}$",
        "T": "$\ef{\gls{T}, \mathrm{\gls{crs}}}$",
        "A": "$\ef{\gls{A}, \mathrm{\gls{crs}}}$",
        "R_per_oscillator": "$\ef{\po{\gls{R}}, \mathrm{\gls{crs}}}$",
        "T_per_oscillator": "$\ef{\po{\gls{T}}, \mathrm{\gls{crs}}}$",
        "A_per_oscillator": "$\ef{\po{\gls{A}}, \mathrm{\gls{crs}}}$",
        "max": "Max",
        "min": "Min",
    }
)

opt_norm_1 = opt_norm_1.set_index(list(opt_norm_1.columns[0:6]))

opt_norm_1.drop(columns=["period_start", "period_stop"]).to_latex(
    buf=root / r"thesis\LaTeX\chapters\appendices\optimal_norm_1_parameters.tex",
    float_format="{:.3e}".format,
    sparsify=True,
    longtable=True,
    escape=False,
    label="tab:optimal_norm_1_parameters",
    position="ht",
    caption="Table of optimal parameters for \gls{lopc} \gls{fom} enhancement factors relative to \gls{crs}",
)

In [ ]:
opt_norm_2 = pd.read_csv(
    data_path / f"run_{run_number}/optimal_norm_2.csv",
    # index_col=[
    #     "variable",
    #     "maxmin",
    #     "lorentz_line",
    #     "window_radius",
    #     "theta_start",
    #     "theta_stop",
    # ],
)

opt_norm_2["variable"] = opt_norm_2["variable"].astype(variable)

opt_norm_2 = opt_norm_2.sort_values(
    by=["variable", "maxmin", "lorentz_line", "window_radius"]
)

opt_norm_2 = opt_norm_2.rename(
    columns={
        "variable": "\gls{fom}",
        "maxmin": "Max/min",
        "lorentz_line": "Lorentz line",
        "window_radius": "Window radius",
        "theta_start": "\gls{theta}",
        "theta_stop": "$\gls{theta}_i$",
        "excitonic_layer_thickness": "\gls{d_e}",
        "num_periods": "\gls{N_lopc}",
        "period": "\gls{Lambda}",
        "value": "Value",
    }
)

opt_norm_2 = opt_norm_2.replace(
    {
        "R": "$\ef{\gls{R}, \mathrm{\gls{frs}}}$",
        "T": "$\ef{\gls{T}, \mathrm{\gls{frs}}}$",
        "A": "$\ef{\gls{A}, \mathrm{\gls{frs}}}$",
        "R_per_oscillator": "$\ef{\po{\gls{R}}, \mathrm{\gls{frs}}}$",
        "T_per_oscillator": "$\ef{\po{\gls{T}}, \mathrm{\gls{frs}}}$",
        "A_per_oscillator": "$\ef{\po{\gls{A}}, \mathrm{\gls{frs}}}$",
        "max": "Max",
        "min": "Min",
    }
)

opt_norm_2 = opt_norm_2.set_index(list(opt_norm_2.columns[0:6]))

opt_norm_2.drop(columns=["period_start", "period_stop"]).to_latex(
    buf=root / r"thesis\LaTeX\chapters\appendices\optimal_norm_2_parameters.tex",
    float_format="{:.3e}".format,
    sparsify=True,
    longtable=True,
    escape=False,
    label="tab:optimal_norm_2_parameters",
    position="ht",
    caption="Table of optimal parameters for \gls{lopc} \gls{fom} enhancement factors relative to \gls{frs}",
)

## Optimal structures for various parameters

### Investigation and notes

In [ ]:
periods = restacked_ds.period.values[1:-1]
# don't set the maximum period to be lower than the maximum excitonic layer thickness, or it will break the plot!
safe_periods = restacked_ds.period.sel(
    period=slice(restacked_ds.excitonic_layer_thickness.values[-1], None)
).values

kdims = [
    hv.Dimension("variable", values=restacked_ds.data_vars),
    hv.Dimension("dataset", values=["restacked_ds", "restacked_diff_1"]),
    hv.Dimension("optimise", values=["max", "min"]),
    hv.Dimension("lorentz_line", range=(-15, 8), default=0),
    hv.Dimension("window_radius", range=(0, 8), default=0),
    hv.Dimension("theta", range=(0, 86), default=0),
    hv.Dimension(
        "cmap", values=["viridis", "cividis", "inferno", "PRGn", "PuOr_r", "RdBu_r"]
    ),
    hv.Dimension("period_start", values=periods, default=periods[0]),
    hv.Dimension("period_stop", values=safe_periods, default=periods[-1]),
    hv.Dimension("integrate_angle", range=(0, 86), default=0),
]

angle_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_plot,
        extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    ),
    kdims=kdims,
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)
# if I'm just looking at normal incidence the extra plots are redundant
dmap = hv.DynamicMap(
    partial(wrapped_2D_plot, extra_plots=["RTA_int", "norm_1_int"], integrate_angle=0),
    kdims=kdims[:9],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# # there is some sort of horrible bug that makes the title and axis on the second plot sometime disappear, along with one of the vlines
# angle_dmap

In [ ]:
# # Use this to record interesting key sets
# angle_dmap.current_key

### Example of how the optimality surfaces were created

In [ ]:
# example plot:
dmap[("R", "restacked_ds", "max", -2, 0, 0, "viridis", 10, 450)]

In [ ]:
# pixel from the example plot: d_e=60, N=40
excitonic_layer_thickness = 60
num_periods = 40
theta = 0
min_period = excitonic_layer_thickness
max_period = excitonic_layer_thickness + int(ds.passive_layer_thickness.max())

coords = {
    "excitonic_layer_thickness": excitonic_layer_thickness,
    "num_periods": num_periods,
    "theta": theta,
}

temp_ds = restacked_ds.sel(**coords).squeeze()
the_plot = (
    plot_R(dataset=temp_ds, x="wavelength", y="period")
    * lorentz_vlines([0], scale=1e-9).opts(opts.VLine(line_dash="dotted"))
    * lorentz_vlines([-2], scale=1e-9).opts(opts.VLine(line_dash="dashdot"))
)

In [ ]:
the_plot = the_plot.redim(period=period_dim)

In [ ]:
the_plot.opts(
    opts.Image(ylim=(min_period, max_period), title=""),
    opts.VLine(line_color="white")
)

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_LOPC_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
print(f"R_LOPC_2D_wl_P_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}")

In [ ]:
# 1D slice along dashdot line
temp_ds_2 = temp_ds.sel(wavelength=lorentz_line_wavelength(-2)/1e-9, method="nearest")
the_plot = plot_R(dataset=temp_ds_2, x="period")

In [ ]:
lorentz_line_wavelength(-2)/1e-9

In [ ]:
the_plot = the_plot.redim(period=period_dim, R="R")

In [ ]:
max_pos = max_min_pos(temp_ds_2, "R", "period")
max_cross = hv.Points((float(max_pos["R_max_period"]), float(max_pos["R_max"])), kdims=[period_dim, "R"])

In [ ]:
the_plot = the_plot * max_cross

In [ ]:
the_plot.opts(
    opts.Curve(title="", ylim=(0, None)),
    opts.Points(color=red, size=5),
    opts.Overlay(show_legend=False)
)

In [ ]:
hv.save(the_plot, filename=fig_path/f"R_LOPC_1D_P_wlL-2_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}", fmt="png", toolbar=None)

In [ ]:
print(f"R_LOPC_1D_P_wlL-2_elt{excitonic_layer_thickness}_N{num_periods}_th{theta}")

### Optimal structures for reflectance

#### Maximising at a specific wavelength

Carniglia and Apfel (1979) showed that the theoretical limit of the reflectance of a multilayer comprising one absorbing and one non-absorbing layer is unity. However, this requires variable layer thicknesses, unlike in our case where the periodicity is strict. As a result, our situation is more like Koppelmann's (1960), with a saturating reflectance depending on the absorption properties at a particular wavelength, although Koppelmann showed this for quarter-wave stacks only and we do not attempt to extend the analytical proof to other periodic multilayers. The gist of the argument is that without careful selection of layer thicknesses to optimise the reflectance with each additional layer pair, a certain amount of light intensity is unavoidably lost to absorption.

Generally, the reflectance near the oscillator resonance peaks at <50 layers (the maximum simulated), while the reflectance far from the resonance peaks at >=50 layers.

High reflectance at a wavelength comes at a cost to transmittance and absorptance locally, but is usually associated with an increase in absorptance at longer wavelengths.

In [ ]:
dmap[(("R", "restacked_ds", "max", 0, 0, 0, "viridis", 10, 450))]

In [ ]:
# dmap[("R", "restacked_ds", "max", -2, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "max", 2, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "max", -5, 0, 0, "viridis", 10, 450)]

In [ ]:
dmap[("R", "restacked_ds", "max", 5, 0, 0, "viridis", 10, 450)]

In [ ]:
# # this calculation is just for the purposes of looking at how reflectance can decrease with period, see above sections
# dmap[("R", "restacked_ds", "max", -7, 0, 0, "viridis", 10, 450)]

#### Maximising over a window of 4 linewidths

When optimising for reflectance over a window of wavelengths (in this case a radius of 2 oscillator linewidths), the optimal number of periods is lower than when optimising for only the central wavelength of that window. This is because with more periods the PBG tends to increase in height but decrease in width. Unlike with purely non-absorbing layers, where the width of the PBG starts to increase again when the reflectance is saturated at 100%, the competing increase in absorption with additional layers means the reflectance peaks at a finite number of periods.

It is particularly difficult to attain a high reflectance in the low-RI contrast region immediately on the blue side of the LO resonance, and the peak values attainable on the lower-contrast blue side of the resonance are generally lower than those attainable on the higher-contrast red side.

It's worth noting that, as is always the case when referring to optical parameters over a range of wavelengths or angles, that this is an integration of the *intrinsic* optical properties, and the *extrinsic* value will depend on the spectral and angular distribution of illumination.

In [ ]:
dmap[(("R", "restacked_ds", "max", 0, 2, 0, "viridis", 10, 450))]

In [ ]:
# dmap[("R", "restacked_ds", "max", -2, 2, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "max", 2, 2, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "max", -5, 2, 0, "viridis", 10, 450)]

In [ ]:
dmap[("R", "restacked_ds", "max", 5, 2, 0, "viridis", 10, 450)]

#### Minimising reflectance

Interestingly, the reflectance at the LO resonance is lower with five layers at 10 nm than a single layer. In this regime, the absorptance is enhanced compared to the CRS at all wavelengths. It is also possible to minimise the reflectance at wavelengths far from the resonance with even higher numbers of thin (10 nm) layers. However, the absolute difference in reflectance with the CRS is generally negligible, and so this is not that relevant for iridoplast-type scenarios. Additionally, the optimal structure degenerates to a pair of thin layers everywhere when a window of radius >=2 linewidths is considered.

In [ ]:
# dmap[("R", "restacked_ds", "min", 0, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", -8, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 0, 2, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 0, 5, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", -8, 1, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 4, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", -5, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", -5, 2, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", -5, 5, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 5, 0, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 5, 2, 0, "viridis", 10, 450)]

In [ ]:
# dmap[("R", "restacked_ds", "min", 5, 5, 0, "viridis", 10, 450)]

#### Angle behaviour

The period which gives the highest overall reflectance at any given wavelength, when integrated over a angular range, is (not strictly) greater than the optimal period for normal incidence. This is because the PBG blueshifts with increasing $\theta$. The overall reflectance values after integration and normalisation are lower than those at normal incidence. Also, there is less specificity in where the reflectance is - generally there is a 'tail' of reflectance on the blue side of the target region.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R", "restacked_ds", "max", 0, 0, 0, "viridis", 10, 450, 45)]

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=250,
#     excitonic_layer_thickness=70,
#     num_periods=50,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([0], scale=1e-9).opts(opts.VLine(line_dash="dotted", color="white"))

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=250,
#     excitonic_layer_thickness=40,
#     num_periods=40,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([0], scale=1e-9).opts(opts.VLine(line_dash="dotted", color="white"))

Supporting evidence: the period which gives the optimal reflectance near the resonance at 30 degrees is higher than that for normal incidence. Also notable is that the number of layers is much higher.

In [ ]:
angle_dmap[("R", "restacked_ds", "max", 0, 2, 30, "viridis", 10, 450, 0)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 290
excitonic_layer_thickness = 60
num_periods = 50

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA(
        {"theta": 30, "label_override": " (θ = 30)"},
        {"theta": 0, "label_override": " (θ = 0)"},
        include=["LOPC"],
        **params
    )
    * lorentz_vlines([-2, 2], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
    * hv.VSpan(lorentz_line_wavelength(-2) * 1e9, lorentz_line_wavelength(2) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_R_max_0_2_30_0_RTA_c", fmt="png", toolbar=False)

In [ ]:
# angle_dmap[("R", "restacked_ds", "max", 0, 2, 60, "viridis", 10, 450, 0)]

##### Consider the case below, where the PBG at normal incidence is clearly redshifted compared to the wavelength range we are trying to optimise reflectance in (-2<2). As the 2D plot below shows, this is because the reflectance is higher in that region overall taking into account the blueshift with angle.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("R", "restacked_ds", "max", 0, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=260,
#     excitonic_layer_thickness=80,
#     num_periods=25,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-2, 2], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=250,
#     excitonic_layer_thickness=70,
#     num_periods=30,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-2, 2], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # I'm curious - what happens if I try to make this PBG better with more layers? --Ah, it doesn't get better because of the absorption, I see.
# plot_RTA_2D(
#     period=250,
#     excitonic_layer_thickness=70,
#     num_periods=50,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-2, 2], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 260
excitonic_layer_thickness = 70
num_periods = 30

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([-2, 2], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
* hv.VSpan(lorentz_line_wavelength(-2) * 1e9, lorentz_line_wavelength(2) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_R_max_0_2_0_45_RTA_c", fmt="png", toolbar=False)

##### As above, but the range (-4<0). Note how hard it is to get any appreciable reflectance here!

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R", "restacked_ds", "max", -2, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=250,
#     excitonic_layer_thickness=80,
#     num_periods=50,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-4, 0], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=230,
#     excitonic_layer_thickness=80,
#     num_periods=40,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-4, 0], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

##### As above, but the range (0, 4).

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R", "restacked_ds", "max", 2, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=270,
#     excitonic_layer_thickness=100,
#     num_periods=20,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([0, 4], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=260,
#     excitonic_layer_thickness=80,
#     num_periods=26,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([0, 4], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

##### As above, but the range (-7, -3).

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R", "restacked_ds", "max", -5, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=220,
#     excitonic_layer_thickness=100,
#     num_periods=40,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-7, -3], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=210,
#     excitonic_layer_thickness=100,
#     num_periods=50,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([-7, -3], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

##### As above, but the range (3, 7). Much higher potential reflectance here.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("R", "restacked_ds", "max", 5, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 290
excitonic_layer_thickness = 120
num_periods = 23

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([3, 7], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
    * hv.VSpan(lorentz_line_wavelength(3) * 1e9, lorentz_line_wavelength(7) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_R_max_5_2_0_45_RTA_c", fmt="png", toolbar=False)

In [ ]:
# # params that optimise reflectance integrated over theta
# plot_RTA_2D(
#     period=290,
#     excitonic_layer_thickness=130,
#     num_periods=21,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([3, 7], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

In [ ]:
# # params that optimise reflectance at normal incidence
# plot_RTA_2D(
#     period=280,
#     excitonic_layer_thickness=110,
#     num_periods=22,
#     theta=(0, 75),
#     include=["LOPC"],
# ) * lorentz_vlines([3, 7], scale=1e-9).opts(
#     opts.VLine(line_dash="dotted", color="white")
# )

### Optimal structures for transmittance

#### Maximum

The optimum structure for maximal transmittance at a wavelength is obvious (a single thin layer). What is more interesting is the way that the maximal transmittance drops off quickly with total excitonic layer thickness. The capacity of the LOPC to increase transmittance is minimal - see below plot with iridoplast-like parameters. It is at its best with a higher number of thin layers (examine this further when looking in to tunability).

In [ ]:
# dmap[("T", "restacked_ds", "max", 0, 0, 0, "cividis", 10, 450)]

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 8}
# lorentz_line_number = 0
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params)

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 8}
# lorentz_line_number = -5
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params)

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 100, "num_periods": 8}
# lorentz_line_number = 0
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params)

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 30}
# lorentz_line_number = 0
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params)

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 20, "num_periods": 30}
# lorentz_line_number = 5
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params).opts(legend_position="right")

In [ ]:
# params = {"theta": 0, "excitonic_layer_thickness": 100, "num_periods": 30}
# lorentz_line_number = 0
# window_radius = 0
# optimum = find_optimum_coords(
#     da=select_lorentz_line(
#         restacked_ds.T.sel(**params, method="nearest"),
#         lorentz_line_number,
#         window_radius,
#     ),
#     dim="period",
# )
# plot_RTA(period=float(optimum.period), **params)

#### Minimum

Where it is possible to, the dominant strategy is to layer up so much material that almost all the light is absorbed. Where it is not possible, the dominant strategy still involves the maximum number of layers, but the thickness of the layers may be reduced to form a higher-quality PBG. This can especially be the case when considering a wider window. Generally, the LOPC has the capacity to reduce transmittance relative to the CRS.

In [ ]:
# dmap[("T", "restacked_ds", "min", 0, 0, 0, "cividis", 10, 450)]

In [ ]:
dmap[("T", "restacked_ds", "min", 0, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_ds", "min", -5, 0, 0, "cividis", 10, 450)]

In [ ]:
dmap[("T", "restacked_ds", "min", -5, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_ds", "min", 5, 0, 0, "cividis", 10, 450)]

In [ ]:
dmap[("T", "restacked_ds", "min", 5, 2, 0, "cividis", 10, 450)]

In [ ]:
# # limit the total excitonic thickness so that absorption doesn't saturate
# temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
#     excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20)
# )
# plots = wrapped_2D_plot("T", temp_ds, "min", 0, 0, 0, "cividis", 10, 450, dim="period", save_figs=False)
# display(plots)

In [ ]:
hv.save(plots[1].options(legend_position="right"), filename=fig_path/"opt_T_min_0_0_0_0_RTA_i_elt20_N20", fmt="png", toolbar=False)

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
    excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20)
)
plots = wrapped_2D_plot("T", temp_ds, "min", 0, 1, 0, "cividis", 10, 450, dim="period", save_figs=False)
display(plots)

In [ ]:
hv.save(plots[1].options(legend_position="right"), filename=fig_path/"opt_T_min_0_1_0_0_RTA_i_elt20_N20", fmt="png", toolbar=False)

In [ ]:
# limit the total excitonic thickness so that absorption doesn't saturate
temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
    excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20)
)
plots = wrapped_2D_plot("T", temp_ds, "min", 0, 2, 0, "cividis", 10, 450, dim="period", save_figs=False)
display(plots)

In [ ]:
hv.save(plots[1].options(legend_position="right"), filename=fig_path/"opt_T_min_0_2_0_0_RTA_i_elt20_N20", fmt="png", toolbar=False)

In [ ]:
# # limit the total excitonic thickness so that absorption doesn't saturate
# temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
#     excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20)
# )
# plots = wrapped_2D_plot("T", temp_ds, "min", 0, 5, 0, "cividis", 10, 450, dim="period", save_figs=False)
# display(plots)

In [ ]:
# hv.save(plots[1].options(legend_position="right"), filename=fig_path/"opt_T_min_0_5_0_0_RTA_i_elt20_N20", fmt="png", toolbar=False)

#### Angle behaviour

##### As expected, the optimal structure for minimising transmittance near the resonance doesn't really change at higher angles, because absorption is isotropic.

In [ ]:
# angle_dmap[("T", "restacked_ds", "min", 0, 2, 30, "cividis", 10, 450, 0)]

Far from the resonance, the reflectance plays a role. As expected, the optimal period is higher than at normal incidence.

In [ ]:
# angle_dmap[("T", "restacked_ds", "min", -5, 2, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_ds", "min", 5, 2, 30, "cividis", 10, 450, 0)]

##### When integrating...

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_ds", "min", 0, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_ds", "min", -5, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_ds", "min", 5, 2, 0, "cividis", 10, 450, 45)]

### Optimal structures for absorptance

#### Maximum

If it is possible to just pile up layers to approach 100% absorptance, that is the dominant strategy. There is a secondary effect in minimising reflectance.

If it is not possible, then slow light enhancement can play a role in increasing absorptance, both at the LO resonance and further away. Additionally, the advantage of the photonic effect can be competitive with just increasing layer thickness (see the plot at -8 Lorentz linewidths, where high absorptance is possible at an excitonic layer thickness of 100 nm as well as 150 nm). However, photonic effects play a smaller role when considering larger integration windows, because the range of the slow light absorption enhancement is limited.

Interestingly, when maximising the absorptance near the resonance, the PBG may be located quite far away (~100 nm) from the target wavelength for enhancement.

In [ ]:
dmap[("A", "restacked_ds", "max", 0, 0, 0, "inferno", 10, 450)]

In [ ]:
dmap[("A", "restacked_ds", "max", 0, 5, 0, "inferno", 10, 450)]

In [ ]:
# # limit the total excitonic thickness so that absorption doesn't saturate
# temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
#     excitonic_layer_thickness=slice(None, 20), num_periods=slice(None, 20)
# )
# wrapped_2D_plot(
#     "A",
#     temp_ds,
#     "max",
#     0,
#     0,
#     0,
#     "inferno",
#     10,
#     450,
#     dim="period",
#     extra_plots=["RTA_int", "norm_1_int"],
# )

In [ ]:
dmap[("A", "restacked_ds", "max", -8, 0, 0, "inferno", 10, 450)]

In [ ]:
# # limit the total excitonic thickness so that absorption doesn't saturate
# temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
#     excitonic_layer_thickness=slice(None, 140), num_periods=slice(None, None)
# )
# wrapped_2D_plot(
#     "A",
#     temp_ds,
#     "max",
#     -8,
#     0,
#     0,
#     "inferno",
#     10,
#     450,
#     dim="period",
#     extra_plots=["RTA_int", "norm_1_int"],
# )

It's interesting that the second-order PBG at 400 nm is more effective at enhancing absorption than the first-order one at 190 nm.

In [ ]:
# # limit the total excitonic thickness so that absorption doesn't saturate
# temp_ds = restacked_ds.drop_sel(excitonic_layer_thickness=0).sel(
#     excitonic_layer_thickness=slice(None, 140), num_periods=slice(None, None)
# )
# wrapped_2D_plot(
#     "A",
#     temp_ds,
#     "max",
#     -8,
#     0,
#     0,
#     "inferno",
#     10,
#     300,
#     dim="period",
#     extra_plots=["RTA_int", "norm_1_int"],
# )

In [ ]:
# dmap[("A", "restacked_ds", "max", 8, 0, 0, "inferno", 10, 450)]

In [ ]:
dmap[("A", "restacked_ds", "max", -8, 2, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_ds", "max", 5, 2, 0, "inferno", 10, 450)]

#### Minimum

Obviously, the minimal absorptance comes with the smallest structures. There isn't much interesting to say about this.

In [ ]:
# dmap[("A", "restacked_ds", "min", 0, 0, 0, "inferno", 10, 450)]

#### Angle behaviour

##### At higher angles, the optimal period is higher. The PBG can reduce absorptance at normal incidence while increasing it at 30 degrees.

In [ ]:
# angle_dmap[("A", "restacked_ds", "max", 0, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_ds", "max", -5, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_ds", "max", 5, 2, 30, "inferno", 10, 450, 0)]

##### When integrating, the capacity for photonic effects to meaningfully enhance absorptance overall is very low, because of their localised nature.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_ds", "max", 0, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_ds", "max", -5, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_ds", "max", 5, 2, 0, "inferno", 10, 450, 45)]

### General note on per-oscillator metrics

When the metric increases in absolute terms with layer thickness (e.g. reflectance, absorptance), I expect at the high-thickness limit for there to be an inverse relationship between that metric on a per-oscillator basis and the total number of oscillators (equivalent up to normalisation with total excitonic layer thickness). The per-oscillator metric will also be low at low layer thicknesses, so the optimum is expected at intermediate values.

What affects the position of the optimum in parameter space, and how, is an open question,

### Optimal structures for reflectance per oscillator

#### Maximum

Reflectance per oscillator is kind of as expected. At low total excitonic layer thickness, there is little reflectance, and at high total excitonic layer thickness the reflectance saturates while the number of oscillators increases without bound, so the peak in reflectance per oscillator is at intermediate values of excitonic layer thickness and number of layers. The interesting thing is that for the LO resonance wavelength, the optimum in this parameter ($16 \times 20\ \text{nm} = 320\ \text{nm}$) is in a similar range to the real iridoplast structure.

The behaviour around the LO resonance is assymmetric; there is a shift to significantly higher total excitonic layer thickness at the -2 Lorentz line ($26 \times 40\ \text{nm} = 1040\ \text{nm}$), but not at the +2 Lorentz line ($14 \times 30\ \text{nm} = 420\ \text{nm}$). Generally, the maximum achievable reflectance per oscillator is lower on the blue (low contrast) side of the resonance than on the red (high contrast) side, which holds true up to at least 8 linewidths away. The total excitonic layer thickness that gives the maximal reflectance per oscillator tends to increase with distance from the LO resonance wavelength.

**It would be cool to plot the maximum R_per_oscillator and corresponding total excitonic layer thickness for each Lorentz line.**

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("R_per_oscillator", "restacked_ds", "max", 0, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", -2, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", 2, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", -5, 0, 0, "viridis", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("R_per_oscillator", "restacked_ds", "max", 5, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", -8, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", 8, 0, 0, "viridis", 10, 450)]

#### Maximum over a window

Taking a larger integration window pushes the optimum towards higher layer thicknesses but lower numbers of layers, leading to a wider and flatter PBG. The optimal PBG can't cover more than about 10 linewidths on the red (high contrast) side or 6 linewidths on the blue (low contrast) side.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("R_per_oscillator", "restacked_ds", "max", 0, 2, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", 0, 5, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", 0, 8, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[('R_per_oscillator', 'restacked_ds', 'max', -7, 3, 0, 'viridis', 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[('R_per_oscillator', 'restacked_ds', 'max', 3, 5, 0, 'viridis', 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "max", -5, 2, 0, "viridis", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("R_per_oscillator", "restacked_ds", "max", 5, 2, 0, "viridis", 10, 450)]

#### Minimum

Obviously, minimising the reflectance per oscillator produces uninteresting structures that typically have low layer thickness and high numbers of layers.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", 0, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", -5, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", 5, 0, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", 0, 2, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", -5, 2, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("R_per_oscillator", "restacked_ds", "min", 5, 2, 0, "viridis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "min", 0, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("R_per_oscillator", "restacked_ds", "min", -5, 2, 0, "viridis", 10, 450, 45)
# ]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "min", 5, 2, 0, "viridis", 10, 450, 45)]

#### Angle behaviour

##### At 30 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "max", 0, 2, 30, "viridis", 10, 450, 0)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("R_per_oscillator", "restacked_ds", "max", -5, 2, 30, "viridis", 10, 450, 0)
# ]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "max", 5, 2, 30, "viridis", 10, 450, 0)]

##### Integrating (angle only)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "max", 0, 0, 0, "viridis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("R_per_oscillator", "restacked_ds", "max", -5, 0, 0, "viridis", 10, 450, 45)
# ]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("R_per_oscillator", "restacked_ds", "max", 5, 0, 0, "viridis", 10, 450, 45)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 290
excitonic_layer_thickness = 80
num_periods = 8

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([5], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Rpo_max_5_0_0_45_RTA_c", fmt="png", toolbar=False)

##### Integrating (angle and wavelength)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "max", 0, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("R_per_oscillator", "restacked_ds", "max", -5, 2, 0, "viridis", 10, 450, 45)
# ]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("R_per_oscillator", "restacked_ds", "max", 5, 2, 0, "viridis", 10, 450, 45)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 290
excitonic_layer_thickness = 90
num_periods = 6

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([3, 7], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
    * hv.VSpan(lorentz_line_wavelength(3) * 1e9, lorentz_line_wavelength(7) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Rpo_max_5_2_0_45_RTA_c", fmt="png", toolbar=False)

##### Integrating (angle and *wide* wavelength)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("R_per_oscillator", "restacked_ds", "max", 0, 8, 0, "viridis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("R_per_oscillator", "restacked_ds", "max", -5, 8, 0, "viridis", 10, 450, 45)
# ]

### Optimal structures for transmittance per oscillator

#### Maximum

Obviously, the maximum transmittance per oscillator occurs when there is the minimum possible amount of absorbing material present.

#### Minimum

Obviously, the minimum transmittance per oscillator occurs when there is lots of absorbing material present. However, even near the resonance, photonic effects can play a role with a wider window. Far from the resonance, photonic effects dominate over intrinsic absorptance.

What is not entirely obvious is that minimising transmittance per oscillator is *not* the same as maximising the sum of (reflectance per oscillator) and (absorptance per oscillator). The latter will tend to produce optima in the intermediate region. I might get onto doing calculations of this at some point, but it's not as useful as it may appear because in a real situation, you would need to know how to weight the relative importance of these two metrics.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("T_per_oscillator", "restacked_ds", "min", 0, 5, 0, "cividis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("T_per_oscillator", "restacked_ds", "min", -8, 0, 0, "cividis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("T_per_oscillator", "restacked_ds", "min", -8, 6, 0, "cividis", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("T_per_oscillator", "restacked_ds", "min", 4, 4, 0, "cividis", 10, 450)]

### Optimal structures for absorptance per oscillator

#### Maximum

Near the LO resonance, the maximum absorptance per oscillator comes with almost the minimum amount of absorbing material - photonic effects aren't helpful beyond the slight absorption increase that comes from having two layers rather than one. However, further from the resonance (at least -4 linewidths below or 2 above) there is the possibility of improving absorptance per oscillator using the photonic structure.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 0, 0, 0, "inferno", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("A_per_oscillator", "restacked_ds", "max", -5, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", -8, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 2, 0, 0, "inferno", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("A_per_oscillator", "restacked_ds", "max", 5, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 8, 0, 0, "inferno", 10, 450)]

#### Maximum over a window

The absorptance per oscillator is still not improved over window centred on the resonance, even when the window includes wavelengths where a photonic enhancement is possible. Further from the resonance, widening the window generally moves the optimum to higher layer thicknesses and lower numbers of periods, similarly to reflectance per oscillator. This is related to the idea of widening the range of photonic effects at the expense of their magnitude.

The photonic enhancement 'washes out' faster on the blue side of the resonance, because the region of absorption enhancement due to the slow light effect doesn't extend very far with low RI contrast. On the red side, the photonic enhancement is evident over wider windows. However, when considering regions very far from the LO resonance, where the intrinsic absorption of the CRS is very low, it is possible for high reflectance to coexist with a (marginally) higher absorptance per oscillator and some degree of absorption enhancement - see plot at -10 linewidths over a window of radius 5.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

# this would be better if the window size was 8
# dmap[('A_per_oscillator', 'restacked_ds', 'max', 0, 4, 0, 'inferno', 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", -4, 1, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", -4, 2, 0, "inferno", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("A_per_oscillator", "restacked_ds", "max", -5, 2, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", -8, 1, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", -10, 5, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 4, 3, 0, "inferno", 10, 450)]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

dmap[("A_per_oscillator", "restacked_ds", "max", 5, 2, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 6, 2, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "max", 7, 1, 0, "inferno", 10, 450)]

#### Minimum

To minimise the absorptance per oscillator, the strategy is to maximise the amount of material and lay a PBG over the target wavelength range. The reduction in absorptance caused by the PBG can be more significant than the reduction in oscillator number from having thicker layers, but the number of layers is always high.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "min", 0, 2, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "min", -5, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "min", 5, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "min", -8, 0, 0, "inferno", 10, 450)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# dmap[("A_per_oscillator", "restacked_ds", "min", 8, 0, 0, "inferno", 10, 450)]

#### Angle behaviour

##### At 30 degrees

The optimal period is higher than at normal incidence.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A_per_oscillator", "restacked_ds", "max", 0, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("A_per_oscillator", "restacked_ds", "max", -5, 2, 30, "inferno", 10, 450, 0)
# ]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("A_per_oscillator", "restacked_ds", "max", 5, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 290
excitonic_layer_thickness = 50
num_periods = 11

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA(
        {"theta": 30, "label_override": " (θ = 30)"},
        {"theta": 0, "label_override": " (θ = 0)"},
        include=["LOPC"],
        **params
    )
    * lorentz_vlines([3, 7], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
    * hv.VSpan(lorentz_line_wavelength(3) * 1e9, lorentz_line_wavelength(7) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Apo_max_5_2_30_0_RTA_c", fmt="png", toolbar=False)

##### Integrating

The photonic advantage disappears because it is localised to a particular angle/wavelength range.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A_per_oscillator", "restacked_ds", "max", 0, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[
#     ("A_per_oscillator", "restacked_ds", "max", -5, 2, 0, "inferno", 10, 450, 45)
# ]

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("A_per_oscillator", "restacked_ds", "max", 5, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 270
excitonic_layer_thickness = 60
num_periods = 17

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([5], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Apo_max_5_0_0_45_RTA_c", fmt="png", toolbar=False)

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

angle_dmap[("A_per_oscillator", "restacked_ds", "max", 5, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 260
excitonic_layer_thickness = 40
num_periods = 8

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([3, 7], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
    * hv.VSpan(lorentz_line_wavelength(3) * 1e9, lorentz_line_wavelength(7) * 1e9)
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.VSpan(color="gray", alpha=0.1),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Apo_max_5_2_0_45_RTA_c", fmt="png", toolbar=False)

## Difference factors

### Optimisation

Can just reuse the dmaps from the RTA section, because difference factors integrate perfectly well.

In [ ]:
# # Use this to record interesting key sets
# angle_dmap.current_key

### Transmittance difference factor

#### Max (near)

##### Normal incidence

At normal incidence and on resonance, optimal structure for maximising absorptance difference factor is similar to for absorptance enhancement factor. As the window width increases, the optimal total excitonic layer thickness also increases. The relevance of the slow light enhancement decreases relative to that of path length enhancement (see 0-5-0-0).

In [ ]:
# dmap[("T", "restacked_diff_1", "max", 0, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "max", 0, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "max", 0, 5, 0, "cividis", 10, 450)]

##### 30 degrees

Higher angle = higher period, otherwise structure is similar.

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", 0, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", 0, 2, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", 0, 5, 30, "cividis", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", 0, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", 0, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", 0, 5, 0, "cividis", 10, 450, 45)]

#### Max (far)

##### Normal incidence

In [ ]:
# dmap[("T", "restacked_diff_1", "max", -5, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "max", -5, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "max", 5, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "max", 5, 2, 0, "cividis", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", -5, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", -5, 2, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", 5, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "max", 5, 2, 30, "cividis", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", -5, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", -5, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", 5, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "max", 5, 2, 0, "cividis", 10, 450, 45)]

#### Min (near)

##### Normal incidence

In [ ]:
# dmap[("T", "restacked_diff_1", "min", 0, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "min", 0, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "min", 0, 5, 0, "cividis", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", 0, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", 0, 2, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", 0, 5, 30, "cividis", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", 0, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", 0, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", 0, 5, 0, "cividis", 10, 450, 45)]

#### Min (far)

##### Normal incidence

In [ ]:
# dmap[("T", "restacked_diff_1", "min", -5, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "min", -5, 2, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "min", 5, 0, 0, "cividis", 10, 450)]

In [ ]:
# dmap[("T", "restacked_diff_1", "min", 5, 2, 0, "cividis", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", -5, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", -5, 2, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", 5, 0, 30, "cividis", 10, 450, 0)]

In [ ]:
# angle_dmap[("T", "restacked_diff_1", "min", 5, 2, 30, "cividis", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", -5, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", -5, 2, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", 5, 0, 0, "cividis", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("T", "restacked_diff_1", "min", 5, 2, 0, "cividis", 10, 450, 45)]

### Absorptance difference factor

#### Max (near)

##### Normal incidence

At normal incidence and on resonance, optimal structure for maximising absorptance difference factor is similar to for absorptance enhancement factor. As the window width increases, the optimal total excitonic layer thickness also increases. The relevance of the slow light enhancement decreases relative to that of path length enhancement (see 0-5-0-0).

In [ ]:
# dmap[("A", "restacked_diff_1", "max", 0, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "max", 0, 2, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "max", 0, 5, 0, "inferno", 10, 450)]

##### 30 degrees

Higher angle = higher period, otherwise structure is similar.

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", 0, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", 0, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", 0, 5, 30, "inferno", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", 0, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", 0, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", 0, 5, 0, "inferno", 10, 450, 45)]

#### Max (far)

##### Normal incidence

In [ ]:
# dmap[("A", "restacked_diff_1", "max", -5, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "max", -5, 2, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "max", 5, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "max", 5, 2, 0, "inferno", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", -5, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", -5, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", 5, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "max", 5, 2, 30, "inferno", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", -5, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", -5, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", 5, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "max", 5, 2, 0, "inferno", 10, 450, 45)]

#### Min (near)

##### Normal incidence

In [ ]:
# dmap[("A", "restacked_diff_1", "min", 0, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "min", 0, 2, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "min", 0, 5, 0, "inferno", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", 0, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", 0, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", 0, 5, 30, "inferno", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", 0, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", 0, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", 0, 5, 0, "inferno", 10, 450, 45)]

#### Min (far)

##### Normal incidence

In [ ]:
# dmap[("A", "restacked_diff_1", "min", -5, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "min", -5, 2, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "min", 5, 0, 0, "inferno", 10, 450)]

In [ ]:
# dmap[("A", "restacked_diff_1", "min", 5, 2, 0, "inferno", 10, 450)]

##### 30 degrees

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", -5, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", -5, 2, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", 5, 0, 30, "inferno", 10, 450, 0)]

In [ ]:
# angle_dmap[("A", "restacked_diff_1", "min", 5, 2, 30, "inferno", 10, 450, 0)]

##### Integrated 0-45 degrees

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", -5, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", -5, 2, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", 5, 0, 0, "inferno", 10, 450, 45)]

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_dmap[("A", "restacked_diff_1", "min", 5, 2, 0, "inferno", 10, 450, 45)]

## Enhancement factors

### Optimisation

In [ ]:
# def wrapped_2D_ef_plot(
#     variable,
#     dataset,
#     ref,
#     optimise="max",
#     lorentz_line=0,
#     window_radius=0,
#     theta=0,
#     cmap="viridis",
#     period_start=None,
#     period_stop=None,
#     integrate_angle: float = None,
#     extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
#     dim=None,  # automatically assign if dataset recognised
#     save_figs=True,
#     restack_ef=None,
#     common_dim=None,
# ):
#     plots = []

#     # detect string labels
#     if str(dataset) == "ds":
#         # the drop_sel is important for avoiding the most common degenerate cases
#         dataset = ds.drop_sel({"excitonic_layer_thickness": 0})

#     if str(ref) == "crs_1":
#         ref = crs_1
#         common_dim = "total_excitonic_thickness"

#     if str(dataset) == "frs_1":
#         ref = frs_1
#         common_dim = "total_thickness"

#     match dim:  # if restack is the default value, automatically set it
#         case "period":
#             restack_ef = True if restack_ef is None else restack_ef
#         case "passive_layer_thickness":
#             restack_ef = False if restack_ef is None else restack_ef
#     integration_dims = []
#     vline_locs = [0]

#     da = dataset[variable]
#     ra = ref[variable]

#     da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)
#     ra = select_lorentz_line(ra, lorentz_line=lorentz_line, window_radius=window_radius)

#     if window_radius == 0:
#         wavelength = float(da.wavelength)
#         title = f"{optimise.capitalize()}imum {variable} enhancement factor at {wavelength:.0f} nm"
#         if lorentz_line != 0:  # don't put two lines over the resonance
#             vline_locs.append(lorentz_line)
#     else:
#         wavelength_start = float(da.wavelength[0])
#         wavelength_stop = float(da.wavelength[-1])
#         title = f"{optimise.capitalize()}imum integrated {variable} enhancement factor between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
#         vline_locs.append(lorentz_line - window_radius)
#         vline_locs.append(lorentz_line + window_radius)
#         integration_dims.append("wavelength")

#     if integrate_angle:  # integrate angle should be a float
#         th = (theta, integrate_angle)
#         da = da.sel(theta=slice(*th))
#         ra = ra.sel(theta=slice(*th))
#         integration_dims.append("theta")
#     else:  # this includes if integrate_angle==0, which is hacky but fine
#         da = da.sel(theta=theta, method="nearest")
#         ra = ra.sel(theta=theta, method="nearest")
#         th = theta

#     if integration_dims:  # if the list isn't empty
#         da = integrate_da(da, integration_dims, weighting=1, normalisation=None)
#         ra = integrate_da(ra, integration_dims, weighting=1, normalisation=None)

#     norm = enhancement_factor(ds=da, ref=ra, common_dim=common_dim, method="groupby")

#     if (
#         window_radius == 0 and not integrate_angle
#     ):  # if the precalculated norm_1 dataset will suffice
#         norm = norm_1[variable].drop_sel(
#             {"excitonic_layer_thickness": 0}
#         )  # then use it instead
#         norm = select_lorentz_line(
#             norm, lorentz_line=lorentz_line, window_radius=window_radius
#         )
#         norm = norm.sel(theta=theta, method="nearest")

#     match restack_ef:
#         case True:
#             norm = restack_plt_to_period(norm)

#     if period_start < period_stop:
#         norm = norm.sel(period=slice(period_start, period_stop))
#     else:  # otherwise no data is selected and everything breaks
#         norm = norm.sel(period=slice(period_start, None))

#     plot_1, optimum_coords = plot_optimum_over_dim(
#         norm,
#         dim=dim,
#         x="excitonic_layer_thickness",
#         y="num_periods",
#         optimise=optimise,
#     )
#     ############################################# code above is fresh, below is stale
#     P = float(optimum_coords["period"])
#     t = float(optimum_coords["excitonic_layer_thickness"])
#     N = float(optimum_coords["num_periods"])
#     # try:  # this should work if not integrating over theta
#     #     th = float(optimum_coords["theta"])
#     # except:  # probably the problem is that theta doesn't exist because I integrated over it already
#     #     th = (theta, integrate_angle)
#     lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
#         opts.VLine(line_color=green, line_dash="dotted"),
#     )

#     # give the resonance line a special colour
#     lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

#     plot_1.opts(
#         opts.QuadMesh(cmap=cmap, xlabel=r"\(d_e\ \text{(nm)}\)"),
#         opts.Points(color="red"),
#         opts.Overlay(
#             # title=f"{title}\nOptimal period: {P:.0f}",
#             title=""
#         ),
#     )

#     plots.append(plot_1)
#     if save_figs:
#         export_png(
#             hv.render(plot_1.options(toolbar=None)),
#             filename=fig_path
#             / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_2D.png",
#             webdriver=web_driver,
#         )

#     if "RTA_normal" in extra_plots:  # plot RTA at theta=0
#         new_plot = plot_RTA(
#             period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
#         )
#         new_plot *= lorentz_lines

#         plots.append(new_plot)
#         if save_figs:
#             export_png(
#                 hv.render(new_plot.options(legend_position="right", toolbar=None)),
#                 filename=fig_path
#                 / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_0.png",
#                 webdriver=web_driver,
#             )

#     if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
#         new_plot = plot_RTA(
#             period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
#         )
#         new_plot *= lorentz_lines
#         plots.append(new_plot)
#         if save_figs:
#             export_png(
#                 hv.render(new_plot.options(legend_position="right", toolbar=None)),
#                 filename=fig_path
#                 / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_i.png",
#                 webdriver=web_driver,
#             )

#     if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
#         sel = {
#             "period": P,
#             "excitonic_layer_thickness": t,
#             "num_periods": N,
#             "theta": 0,
#         }
#         new_plot = (
#             plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
#         ).opts(opts.Overlay(legend_position="bottom_right"))
#         plots.append(new_plot)
#         if save_figs:
#             export_png(
#                 hv.render(new_plot.options(toolbar=None)),
#                 filename=fig_path
#                 / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_0.png",
#                 webdriver=web_driver,
#             )

#     if (
#         "norm_1_int" in extra_plots
#     ):  # plot enhancement factor at theta OR integrating over theta
#         try:  # this should work if not integrating over theta
#             sel = {
#                 "period": P,
#                 "excitonic_layer_thickness": t,
#                 "num_periods": N,
#                 "theta": th,
#             }
#             new_plot = (
#                 plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
#             ).opts(opts.Overlay(legend_position="bottom_right"))
#         except:  # if integrating, we need to do the integral *before* normalising
#             ds_int = sel_or_integrate(ds, dim="theta", val=th)
#             crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
#             norm = enhancement_factor(
#                 ds_int,
#                 ref=crs_1_int,
#                 common_dim="total_excitonic_thickness",
#                 method="groupby",
#             )
#             restacked_norm = restack_plt_to_period(norm)

#             sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
#             new_plot = (
#                 plot_ef(variable="A", dataset=restacked_norm, sel=sel) * lorentz_lines
#             ).opts(opts.Overlay(legend_position="bottom_right"))
#         plots.append(new_plot)
#         if save_figs:
#             export_png(
#                 hv.render(new_plot.options(toolbar=None)),
#                 filename=fig_path
#                 / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_i.png",
#                 webdriver=web_driver,
#             )

#     return hv.Layout(plots).cols(1)

In [ ]:
def wrapped_2D_ef_plot(
    variable,
    dataset,
    ref,
    optimise="max",
    lorentz_line=0,
    window_radius=0,
    theta=0,
    cmap="viridis",
    period_start=None,
    period_stop=None,
    integrate_angle: float = None,
    extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
    dim=None,  # automatically assign if dataset recognised
    save_figs=True,
    restack_ef=None,
    common_dim=None,
    shade_window=True,
):
    """I freely admit this is spaghetti code."""
    plots = []

    # detect string labels
    if str(dataset) == "ds":
        # the drop_sel is important for avoiding the most common degenerate cases
        dataset = ds.drop_sel({"excitonic_layer_thickness": 0})

    if str(ref) == "crs_1":
        ref = crs_1
        common_dim = "total_excitonic_thickness"
        stored_ds = xr.open_mfdataset(
            data_path / f"run_{run_number}/opt_over_period_norm_1.nc",
            engine=xarray_engine,
            lock=False,
        )
        stored_tab = pd.read_csv(
            data_path / f"run_{run_number}/optimal_norm_1.csv",
            index_col=[
                "variable",
                "maxmin",
                "lorentz_line",
                "window_radius",
                "theta_start",
                "theta_stop",
            ],
        )

    if str(dataset) == "frs_1":
        ref = frs_1
        common_dim = "total_thickness"
        stored_ds = xr.open_mfdataset(
            data_path
            / f"run_{run_number}/opt_over_period_norm_2.nc",  # this doesn't actually exist yet
            engine=xarray_engine,
            lock=False,
        )
        stored_tab = pd.read_csv(
            data_path / f"run_{run_number}/optimal_norm_2.csv",
            index_col=[
                "variable",
                "maxmin",
                "lorentz_line",
                "window_radius",
                "theta_start",
                "theta_stop",
            ],
        )

    match dim:  # if restack is the default value, automatically set it
        case "period":
            restack_ef = True if restack_ef is None else restack_ef
        case "passive_layer_thickness":  # this is not really supported
            restack_ef = False if restack_ef is None else restack_ef
    integration_dims = []
    vline_locs = [0]

    da = dataset[variable]
    ra = ref[variable]

    da = select_lorentz_line(da, lorentz_line=lorentz_line, window_radius=window_radius)
    ra = select_lorentz_line(ra, lorentz_line=lorentz_line, window_radius=window_radius)

    try:
        optimum_coords = stored_tab.loc[
            variable, optimise, lorentz_line, window_radius, theta, integrate_angle
        ]
        plot_1 = (
            stored_ds[variable]
            .sel(
                optimise=optimise,
                lorentz_line=lorentz_line,
                window_radius=window_radius,
                theta_start=theta,
                theta_stop=integrate_angle,
            )
            .hvplot(kind="quadmesh", x="excitonic_layer_thickness", y="num_periods")
        )
        plot_1 *= hv.Points(
            (optimum_coords["excitonic_layer_thickness"], optimum_coords["num_periods"])
        )
    except (KeyError, NameError):
        warnings.warn("stored data not found, computing from scratch")
        if window_radius != 0:
            integration_dims.append("wavelength")

        if integrate_angle:  # integrate angle should be a float
            th = (theta, integrate_angle)
            da = da.sel(theta=slice(*th))
            ra = ra.sel(theta=slice(*th))
            integration_dims.append("theta")
        else:  # this includes if integrate_angle==0, which is hacky but fine
            da = da.sel(theta=theta, method="nearest")
            ra = ra.sel(theta=theta, method="nearest")
            th = theta

        if integration_dims:  # if the list isn't empty
            da = integrate_da(da, integration_dims, weighting=1, normalisation=None)
            ra = integrate_da(ra, integration_dims, weighting=1, normalisation=None)

        norm = enhancement_factor(
            ds=da, ref=ra, common_dim=common_dim, method="groupby"
        )

        if (
            window_radius == 0 and not integrate_angle
        ):  # if the precalculated norm_1 dataset will suffice
            norm = norm_1[variable].drop_sel(
                {"excitonic_layer_thickness": 0}
            )  # then use it instead
            norm = select_lorentz_line(
                norm, lorentz_line=lorentz_line, window_radius=window_radius
            )
            norm = norm.sel(theta=theta, method="nearest")

        match restack_ef:
            case True:
                norm = restack_plt_to_period(norm)

        if period_start < period_stop:
            norm = norm.sel(period=slice(period_start, period_stop))
        else:  # otherwise no data is selected and everything breaks
            norm = norm.sel(period=slice(period_start, None))

        plot_1, optimum_coords = plot_optimum_over_dim(
            norm,
            dim=dim,
            x="excitonic_layer_thickness",
            y="num_periods",
            optimise=optimise,
        )
    finally:
        vline_locs = [0]
        da = select_lorentz_line(
            dataset, lorentz_line=lorentz_line, window_radius=window_radius
        )

        if window_radius == 0:
            wavelength = float(da.wavelength)
            title = f"{optimise.capitalize()}imum {variable} enhancement factor at {wavelength:.0f} nm"
            if lorentz_line != 0:  # don't put two lines over the resonance
                vline_locs.append(lorentz_line)
        else:
            wavelength_start = float(da.wavelength[0])
            wavelength_stop = float(da.wavelength[-1])
            title = f"{optimise.capitalize()}imum integrated {variable} enhancement factor between {wavelength_start:.0f} and {wavelength_stop:.0f} nm"
            vline_locs.append(lorentz_line - window_radius)
            vline_locs.append(lorentz_line + window_radius)

    P = float(optimum_coords["period"])
    t = float(optimum_coords["excitonic_layer_thickness"])
    N = float(optimum_coords["num_periods"])
    th = (theta, integrate_angle) if integrate_angle else theta

    lorentz_lines = lorentz_vlines(vline_locs, scale=1e-9, mode="wavelength").opts(
        opts.VLine(line_color=green, line_dash="dotted"),
    )

    # give the resonance line a special colour
    lorentz_lines.VLine.I.opts(opts.VLine(line_color=yellow))

    # shade the integration window:
    if window_radius != 0 and shade_window:
        window_vspan = hv.VSpan(
            lorentz_lines.VLine.II.x, lorentz_lines.VLine.III.x
        ).opts(
            opts.VSpan(color="gray", alpha=0.1)
        )  # shade the area between the second and third lines
        lorentz_lines *= window_vspan

    plot_1.opts(
        opts.QuadMesh(cmap=cmap, xlabel=r"\(d_e\ \text{(nm)}\)"),
        opts.Points(color="red"),
        opts.Overlay(
            # title=f"{title}\nOptimal period: {P:.0f}",
            title=""
        ),
    )

    plots.append(plot_1)
    if save_figs:
        export_png(
            hv.render(plot_1.options(toolbar=None)),
            filename=fig_path
            / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_2D.png",
            webdriver=web_driver,
        )

    if "RTA_normal" in extra_plots:  # plot RTA at theta=0
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=0
        )
        new_plot *= lorentz_lines

        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(legend_position="right", toolbar=None)),
                filename=fig_path
                / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_0.png",
                webdriver=web_driver,
            )

    if "RTA_int" in extra_plots:  # plot RTA at theta OR integrating over theta
        new_plot = plot_RTA(
            period=P, excitonic_layer_thickness=t, num_periods=N, theta=th
        )
        new_plot *= lorentz_lines
        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(legend_position="right", toolbar=None)),
                filename=fig_path
                / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_RTA_i.png",
                webdriver=web_driver,
            )

    if "norm_1_normal" in extra_plots:  # plot enhancement factor at theta=0
        sel = {
            "period": P,
            "excitonic_layer_thickness": t,
            "num_periods": N,
            "theta": 0,
        }
        new_plot = (
            plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
        ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(toolbar=None)),
                filename=fig_path
                / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_0.png",
                webdriver=web_driver,
            )

    if (
        "norm_1_int" in extra_plots
    ):  # plot enhancement factor at theta OR integrating over theta
        try:  # this should work if not integrating over theta
            sel = {
                "period": P,
                "excitonic_layer_thickness": t,
                "num_periods": N,
                "theta": th,
            }
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm_1, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        except:  # if integrating, we need to do the integral *before* normalising
            ds_int = sel_or_integrate(ds, dim="theta", val=th)
            crs_1_int = sel_or_integrate(crs_1, dim="theta", val=th)
            norm = enhancement_factor(
                ds_int,
                ref=crs_1_int,
                common_dim="total_excitonic_thickness",
                method="groupby",
            )
            restacked_norm = restack_plt_to_period(norm)

            sel = {"period": P, "excitonic_layer_thickness": t, "num_periods": N}
            new_plot = (
                plot_ef(variable="A", dataset=restacked_norm, sel=sel) * lorentz_lines
            ).opts(opts.Overlay(legend_position="bottom_right"))
        plots.append(new_plot)
        if save_figs:
            export_png(
                hv.render(new_plot.options(toolbar=None)),
                filename=fig_path
                / f"opt_{variable}ef_{optimise}_{lorentz_line}_{window_radius}_{theta}_{integrate_angle}_AEF_i.png",
                webdriver=web_driver,
            )

    print(f"optimum parameters: P={P}, t={t}, N={N}")
    return hv.Layout(plots).cols(1)

#### Serious business

In [ ]:
periods = restacked_ds.period.values[1:-1]
# don't set the maximum period to be lower than the maximum excitonic layer thickness, or it will break the plot!
safe_periods = restacked_ds.period.sel(
    period=slice(restacked_ds.excitonic_layer_thickness.values[-1], None)
).values

kdims = [
    hv.Dimension("variable", values=restacked_norm_1.data_vars),
    hv.Dimension("dataset", values=["ds"]),
    hv.Dimension("ref", values=["crs_1", "frs_1"]),
    hv.Dimension("optimise", values=["max", "min"]),
    hv.Dimension("lorentz_line", range=(-15, 8), default=0),
    hv.Dimension("window_radius", range=(0, 8), default=0),
    hv.Dimension("theta", range=(0, 86), default=0),
    hv.Dimension(
        "cmap", values=["viridis", "cividis", "inferno", "PRGn", "PuOr_r", "RdBu_r"]
    ),
    hv.Dimension("period_start", values=periods, default=periods[0]),
    hv.Dimension("period_stop", values=safe_periods, default=periods[-1]),
    hv.Dimension("integrate_angle", range=(0, 86), default=0),
]


angle_ef_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_ef_plot,
        extra_plots=["RTA_normal", "RTA_int", "norm_1_normal", "norm_1_int"],
        dim="period",
    ),
    kdims=kdims,
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)
# if I'm just looking at normal incidence the extra plots are redundant
ef_dmap = hv.DynamicMap(
    partial(
        wrapped_2D_ef_plot,
        integrate_angle=0,
        extra_plots=["RTA_int", "norm_1_int"],
        dim="period",
    ),
    kdims=kdims[:10],
).opts(
    opts.Overlay(legend_position="right", legend_opts={"background_fill_alpha": 0.5})
)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# # there is some sort of horrible bug that makes the title and axis on the second plot sometime disappear, along with one of the vlines
# angle_ef_dmap

In [ ]:
# # Use this to record interesting key sets
# angle_ef_dmap.current_key

### Reflectance enhancement factor

The plots of max and min reflectance enhancement factor look quite odd, but ultimately are not that interesting from a design perspective.

In [ ]:
# # This is just the maximum reflectance structure again
# ef_dmap[("R", "ds", "crs_1", "max", 0, 0, 0, "PRGn", 10, 460)]

The band at 110-120 nm is probably due to two things:
1. The reflectance of the CRS getting very low as the total excitonic thickness becomes larger than the wavelength of light, leading to incoherent reflection;
2. That particular excitonic layer thickness makes creating a reflectance-suppressing photonic effect difficult.

In [ ]:
# ef_dmap[("R", "ds", "crs_1", "min", 0, 0, 0, "PRGn", 10, 460)]

In [ ]:
# # why is minimum R enhancement factor so high when excitonic layer thickness is 110 nm?
# wrapped_2D_plot(
#     "R",
#     restacked_norm_1.sel(
#         excitonic_layer_thickness=slice(110, 120), num_periods=slice(8, None)
#     ),
#     "min",
#     0,
#     0,
#     0,
#     "viridis",
#     10,
#     450,
#     dim="period",
# )

### Transmittance enhancement factor

#### Maximum

The result at the resonance is basically meaningless. The maximal values far from the resonance are very small.

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "max", 0, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "max", -2, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "max", 2, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "max", -5, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "max", 5, 0, 0, "PuOr_r", 10, 460)]

#### Minimum

The minimum transmittance enhancement factor at the resonance is surprisingly high. This tells us that the light intensity that is diverted to reflectance comes almost entirely at the cost of absorptance, no matter the parameters. (Really a diff would be better than a norm here.)

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "min", 0, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "min", -2, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "min", 2, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "min", -5, 0, 0, "PuOr_r", 10, 460)]

In [ ]:
# ef_dmap[("T", "ds", "crs_1", "min", 5, 0, 0, "PuOr_r", 10, 460)]

### Absorptance enhancement factor

#### Optimal structure for maximising absorptance enhancement near the resonance

The maximal absorptance enhancement near the LO resonance (between the resonance and -1 linewidth from it) is confined to a narrow band around a total excitonic thickness of roughly 100 nm. The maximum possible enhancement is modest and also not precisely targeted (there is a greater enhancement at longer wavelengths). Furthermore, it is not accompanied by a strong reflectance band, because of the low number of layers.

In [ ]:
ef_dmap[("A", "ds", "crs_1", "max", 0, 0, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(1, None)), clone=True
)

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", 0, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
ef_dmap[("A", "ds", "crs_1", "max", 0, 5, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(1, None)), clone=True
)

#### Optimal structure for maximising absorption enhancement far from the resonance

Beyond a target wavelength of -2 linewidths below or 1 above the resonance, the optimum starts to move, first towards higher numbers of periods, then thicker layers. Bands of local optima in the 2D plot start to appear, as certain sets of parameters have higher potential. (**Why?**)

As we move further from the resonance and towards regions of lower intrinsic absorptance, the maximum attainable wavelength specific absorptance enhancement increases and the optimal structures that produce these become larger. These structures also have higher reflectance - although local optima with lower reflectance also come into play.

These effects are more pronounced on the red (high-contrast) side of the resonance than the blue side.

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", -2, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", -5, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", -5, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", -8, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", 2, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )
# # should do a windowed version of this

In [ ]:
# # example of a local optimum with lower reflectance
# wrapped_2D_plot(
#     "A",
#     restacked_norm_1.sel(excitonic_layer_thickness=slice(20, None)),
#     "max",
#     2,
#     0,
#     0,
#     "inferno",
#     10,
#     450,
#     dim="period",
# )

In [ ]:
ef_dmap[("A", "ds", "crs_1", "max", 5, 0, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(1, None)), clone=True
)

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", 5, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "max", 8, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
ef_dmap[("A", "ds", "crs_1", "max", -8, 5, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(1, None)), clone=True
)

#### Optimal structure for minimising absorptance enhancement

Near the resonance, the structures that minimise absorptance enhancement are almost identical to those maximising reflectance, which tells us that reducing absorptance by enhancing transmittance is not really a thing.

The strategy to get the minimal absorptance enhancement far from the resonance is to saturate the absorptance of the CRS, then add a large reflectance band on top. It looks similar to the optimal structure for reflectance. However, we also see the emergence on local minima at lower total excitonic layer thicknesses.

Again, sometimes the second-order PBG is more effective than the first-order one.

In [ ]:
ef_dmap[("A", "ds", "crs_1", "min", 0, 0, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(None, 1)), clone=True
)

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", 0, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
ef_dmap[("A", "ds", "crs_1", "min", 0, 5, 0, "inferno", 10, 450)].opts(
    opts.QuadMesh(clim=(None, 1)), clone=True
)

In [ ]:
# # Compare this to structure for maximal reflectance at this wavelength
# ef_dmap[("A", "ds", "crs_1", "min", -2, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# # This is exactly the structure for maximal reflectance at this wavelength
# ef_dmap[("A", "ds", "crs_1", "min", 1, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# # This is exactly the structure for maximal reflectance at this wavelength - but note the local minimum at lower thicknesses!
# ef_dmap[("A", "ds", "crs_1", "min", -5, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", -8, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# # Note the difference when we exclude the second-order PBG
# ef_dmap[("A", "ds", "crs_1", "min", -8, 0, 0, "inferno", 10, 400)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# # This is similar to the structure for maximal reflectance at this wavelength
# ef_dmap[("A", "ds", "crs_1", "min", 8, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", -5, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", -5, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", -5, 5, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", 5, 0, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# ef_dmap[("A", "ds", "crs_1", "min", 5, 2, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# # don't trust this - integral extends off the edge of the plot
# ef_dmap[("A", "ds", "crs_1", "min", 5, 5, 0, "inferno", 10, 450)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

#### Angle behaviour

##### At 30 degrees - higher period, otherwise pretty similar to normal incidence

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 0, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", -5, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 5, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", 0, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", -5, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", 5, 2, 30, "inferno", 10, 450, 0)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

##### Integrating (max)

At the resonance, the structure is actually very similar to the case at normal incidence - because the absorption enhancement isn't really coming from a photonic effect, but from the incoherent scattering. The period is slightly higher. The number of layers is higher but the total excitonic thickness is nearly the same. When increasing the wavelength window, the only difference is to increase the number of layers further (thus strengthening the optical path length enhancement at further wavelengths).

Far from resonance, the optimal number of layers and thickness are both much lower after an angle integral, indicating that the slow light enhancement becomes less important and the incoherent scattering more important.

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 0, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 0, 5, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 0, 8, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", -5, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

plots = angle_ef_dmap[("A", "ds", "crs_1", "max", 5, 0, 0, "inferno", 10, 450, 45)].opts(
    opts.QuadMesh(clim=(1, None)), clone=True
)
plots

In [ ]:
new_plots = [(plots[4].relabel(label="integrated", depth=1)), (plots[3].relabel(label="(θ=0)", depth=1))]
the_plot = hv.Overlay(new_plots).opts(opts.Curve(line_dash=hv.Cycle(["solid", "dashed"]), ylabel="γA"), opts.HLine(color=blue), opts.Overlay(legend_position="right"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Aef_max_5_0_0_45_AEF_c", fmt="png", toolbar=False)

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 270
excitonic_layer_thickness = 60
num_periods = 24

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([5], scale=1e-9)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Aef_max_5_0_0_45_RTA_c", fmt="png", toolbar=False)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", -5, 0, 0, "inferno", 10, 450, 75)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 5, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "max", 0, 2, 0, "inferno", 10, 450, 85)].opts(
#     opts.QuadMesh(clim=(1, None)), clone=True
# )

##### Integrating (min)

At resonance, the optimal parameters to minimise the angle-integrated absorptance enhancement factor are basically the same as when not integrating over angle. The period and layer thickness are slightly higher. Far from resonance, again the periods are slightly higher and the excitonic layer thicknesses somewhat higher.

What is quite notable is that when integrating over angle, it becomes very difficult to actually obtain an absorptance enhancement factor <1 for low total excitonic thickness. In other words, a small iridoplast-like structure can’t be built such that it has less absorption than its CRS.

In [ ]:
%%capture --no-stdout --no-display
# ignore divide by zero warnings

plots = angle_ef_dmap[("A", "ds", "crs_1", "min", 0, 0, 0, "inferno", 10, 450, 45)].opts(
    opts.QuadMesh(clim=(None, 1)), clone=True
)
plots

In [ ]:
new_plots = [(plots[4].relabel(label="integrated", depth=1)), (plots[3].relabel(label="(θ=0)", depth=1))]
the_plot = hv.Overlay(new_plots).opts(opts.Curve(line_dash=hv.Cycle(["solid", "dashed"]), ylabel="γA"), opts.HLine(color=blue), opts.Overlay(legend_position="right"))
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Aef_min_0_0_0_45_AEF_c", fmt="png", toolbar=False)

In [ ]:
# comparison of the integrated and unintegrated RTA at normal incidence
# params optimise R when integrated
period = 260
excitonic_layer_thickness = 60
num_periods = 8

params = dict(
    period=period,
    excitonic_layer_thickness=excitonic_layer_thickness,
    num_periods=num_periods,
)

the_plot = (
    compare_RTA_normal_vs_integrated(**params)
    * lorentz_vlines([0], scale=1e-9, label="resonance")
).opts(
    opts.VLine(line_dash="dotted", color=green),
    opts.VLine("resonance", line_dash="dotted", color=yellow),
    opts.Overlay(legend_position="right"),
)
the_plot

In [ ]:
hv.save(the_plot, filename=fig_path/"opt_Aef_min_0_0_0_45_RTA_c", fmt="png", toolbar=False)

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", 0, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", 0, 5, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", -5, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

In [ ]:
# %%capture --no-stdout --no-display
# # ignore divide by zero warnings

# angle_ef_dmap[("A", "ds", "crs_1", "min", 5, 2, 0, "inferno", 10, 450, 45)].opts(
#     opts.QuadMesh(clim=(None, 1)), clone=True
# )

# Cleanup

In [ ]:
# hv.archive.export()